In [ ]:
# ===============================
#  Standard Library
# ===============================
import os
import re
from datetime import datetime, timedelta

# Suppress TensorFlow logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 0=all, 1=INFO, 2=WARNING, 3=ERROR


# ===============================
#  Data Handling
# ===============================
import numpy as np
import pandas as pd


# ===============================
#  Visualization
# ===============================
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import umap
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

# Suppress specific warnings from seaborn
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")


# ===============================
#  Machine Learning & Clustering
# ===============================
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import estimate_bandwidth
from sklearn.cluster import (
    KMeans,
    MiniBatchKMeans,
    Birch,
    DBSCAN,
    AgglomerativeClustering,
    AffinityPropagation,
    MeanShift,
    estimate_bandwidth,
    SpectralClustering,
    OPTICS,
    estimate_bandwidth
)
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE


# ===============================
#  Metrics & Evaluation
# ===============================
from sklearn.metrics import (
    silhouette_score,
    calinski_harabasz_score,
    davies_bouldin_score
)


# ===============================
#  Hierarchical Clustering
# ===============================
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster


# ===============================
#  Utilities
# ===============================
!pip install kneed
from kneed import KneeLocator
import hdbscan

In [ ]:
df = pd.read_csv('online_retail.csv', encoding='ISO-8859-1')

# Task 1
Inspect the dataframe `df` by displaying the first few rows, column names, data types, and summary statistics. Check for and handle missing and duplicate values. Visualize the data distribution and relationships between variables. Summarize the findings.

## Initial data inspection

Display the first few rows, column names, data types, and summary statistics of the dataframe.


In [ ]:
display(df.head())
display(df.info())
display(df.describe())

## Handle missing values

Check for missing values and decide on an appropriate strategy to handle them (e.g., imputation or removal).


In [ ]:
missing_values = df.isnull().sum()
display("Missing values in each column:")
display(missing_values)

In [ ]:
df_cleaned = df.dropna(subset=['Description', 'Customer ID'])
missing_values_after_drop = df_cleaned.isnull().sum()
display("Missing values after dropping rows with missing 'Description' or 'Customer ID':")
display(missing_values_after_drop)

## Handle duplicate values

Check for duplicate values and decide on an appropriate strategy to handle them (e.g., removal).


In [ ]:
duplicate_rows = df_cleaned.duplicated().sum()
display(f"Number of duplicate rows: {duplicate_rows}")

In [ ]:
df_cleaned = df_cleaned.drop_duplicates(keep='first')
duplicate_rows_after_drop = df_cleaned.duplicated().sum()
display(f"Number of duplicate rows after removal: {duplicate_rows_after_drop}")

## Data visualization

Generate relevant visualizations to understand the data distribution and relationships between variables.


In [ ]:
# Set global style
sns.set_theme(style="whitegrid", font_scale=0.9)

# Pre-compute frequently used values
top_countries = df_cleaned['Country'].value_counts().nlargest(10)
top_stock_codes = df_cleaned.groupby('StockCode')['Quantity'].sum().nlargest(10)
sample_df = df_cleaned.sample(n=10000, random_state=42)

# Function to rotate and align x-tick labels
def rotate_xlabels(ax, rotation=90, ha='right'): # Increased rotation to 90
    ax.set_xticks(ax.get_xticks()) # Use FixedLocator by getting current ticks
    ax.set_xticklabels(ax.get_xticklabels(), ha=ha)

# Subplots for Quantity, Price, and Total_Price Histograms
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Distribution of Data', fontsize=16)

# Scatter: Quantity vs Price
sns.scatterplot(x='Quantity', y='Price', data=sample_df, ax=axes[0])
axes[0].set(title='Quantity vs Price (Sampled)', xlabel='Quantity', ylabel='Price', xlim=(-100, 200), ylim=(-10, 100))

# Bar: Top 10 Countries
sns.barplot(
    x=top_countries.index,
    y=top_countries.values,
    hue=top_countries.index,
    palette="viridis",
    legend=False,
    ax=axes[1]
)
axes[1].set(title='Top 10 Countries by Number of Orders', xlabel='Country', ylabel='Number of Orders')
plt.setp(axes[1].get_xticklabels(), rotation=45, ha='right')

# Bar: Top 10 Stock Codes
sns.barplot(
    x=top_stock_codes.index,
    y=top_stock_codes.values,
    hue=top_stock_codes.index,
    palette="mako",
    legend=False,
    ax=axes[2]
)
axes[2].set(title='Top 10 Most Sold Stock Codes by Quantity', xlabel='Stock Code', ylabel='Total Quantity Sold')
plt.setp(axes[2].get_xticklabels(), rotation=45, ha='right')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# Task 2
Analyze and visualize the dataframe `df`. Perform initial data inspection, handle missing values (excluding 'Customer ID'), handle duplicate values, and analyze/merge rows based on 'Invoice', 'StockCode', and 'Customer ID'. Finally, visualize the data and summarize the findings.

## Analyze and potentially merge rows

Group the data by (Invoice, StockCode, Customer ID) and analyze the groups. Decide on a strategy to handle groups with multiple rows (e.g., merging).


In [ ]:
grouped = df_cleaned.groupby(['Invoice', 'StockCode', 'Customer ID']).size().reset_index(name='row_count')
multiple_entries = grouped[grouped['row_count'] > 1]
display("Groups with multiple entries:")
display(multiple_entries.head())

In [ ]:
df_aggregated = df_cleaned.groupby(['Invoice', 'StockCode', 'Description', 'Customer ID', 'Country', 'InvoiceDate']).agg(
    Quantity=('Quantity', 'sum'),
    Price=('Price', 'sum')
).reset_index()

# Verify that there are no more multiple entries for the same key after aggregation
grouped_aggregated = df_aggregated.groupby(['Invoice', 'StockCode', 'Customer ID']).size().reset_index(name='row_count')
multiple_entries_after_aggregation = grouped_aggregated[grouped_aggregated['row_count'] > 1]
display("Groups with multiple entries after aggregation:")
display(multiple_entries_after_aggregation)

In [ ]:
df_aggregated = df_aggregated.drop_duplicates(subset=['Invoice', 'StockCode', 'Customer ID'], keep='first')

# Final verification
grouped_final = df_aggregated.groupby(['Invoice', 'StockCode', 'Customer ID']).size().reset_index(name='row_count')
multiple_entries_final = grouped_final[grouped_final['row_count'] > 1]
display("Groups with multiple entries after final deduplication:")
display(multiple_entries_final)

In [ ]:
df_aggregated.shape

In [ ]:
def cleaner(df):
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='mixed', dayfirst=True)
    print("\n--- Cleaning: Description and StockCode ---")
    for col in ['Description', 'StockCode']:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().str.lower()
            print(f"'{col}' column cleaned (stripped and lowercased).")
        else:
            print(f"'{col}' column not found — skipped.")
    return df

dff = cleaner(df_aggregated)

## Disambiguate Transaction Types

This code defines a function `disambiguate_transaction_types` that classifies transactions into different types ('Sale', 'Internal Adjustment', 'Customer Return') based on the characteristics of the 'Quantity', 'Invoice', and 'Description' columns. It helps in separating sales transactions from returns or internal adjustments for more accurate analysis.

In [ ]:
def disambiguate_transaction_types(df):
    # Lower case invoice and description
    if 'Invoice' in df.columns:
        df['Invoice_cleaned'] = df['Invoice'].astype(str).str.upper().str.strip()
    else:
        raise KeyError("Column 'Invoice' is missing from the DataFrame.")

    if 'Description' in df.columns:
        df['Description_cleaned'] = df['Description'].astype(str).str.lower().str.strip()
    else:
        raise KeyError("Column 'Description' is missing from the DataFrame.")

    # Internal adjustment phrases
    internal_adjustment_phrases = [
        'damages', 'samples/damages', 'damages/display', 'damages?', 'damages/showroom etc',
        'damages/credits from asos.', 'damages/dotcom?', 'damages/samples', 'damages wax',
        '????damages????', 'possible damages or lost?', 'incorrectly made-thrown away.',
        'wrong barcode', 'wrong barcode (22467)', 'sold with wrong barcode',
        'wrongly sold as sets', 'wrongly sold sets', 'wrongly sold (22719) barcode',
        'check', 'stock check', 'check?', '??', '???', '???lost', '?? missing',
        '????missing', '???missing', 'lost??', '???',
        'wet damaged', 'reverse 21/5/10 adjustment', 'adjustment', 'adjust',
        'reverse previous adjustment', 'adjust bad debt', 'taig adjust', 'taig adjust no stock',
        'temp adjustment', 'oops ! adjustment', 'dotcom adjust', 'missing', '?missing',
        'missing?', 'faulty', 're-adjustment'
    ]

    # Default all to 'Sale'
    df['Transaction_Type'] = 'Sale'

    # Internal Adjustments
    df.loc[
        (df['Quantity'] < 0) &
        (~df['Invoice_cleaned'].str.startswith('C')) &
        (df['Description_cleaned'].isin(internal_adjustment_phrases)),
        'Transaction_Type'
    ] = 'Internal Adjustment'

    # Customer Returns
    df.loc[
        (df['Quantity'] < 0) &
        (df['Invoice_cleaned'].str.startswith('C')),
        'Transaction_Type'
    ] = 'Customer Return'

    # 6: Summary
    print("Transaction types classified:")
    print(df['Transaction_Type'].value_counts().to_dict())
    df.drop(columns=['Invoice_cleaned', 'Description_cleaned'], axis=1, inplace=True)

    return df

In [ ]:
dff = disambiguate_transaction_types(dff)
dff.head()

## Categorize Item Types

This code defines a function `categorize_item_types` that assigns each transaction to a specific **item type** (e.g., *Product, Discount, Shipping/Service, Amazon Related, Operational Adjustment*) based on rules applied to the `StockCode` and `Description` fields.

It starts by assuming all items are *Products* and then reclassifies them using keyword matching (such as "postage" → *Shipping/Service*, "discount" → *Discount*, "amazon fee" → *Amazon Related*). It also applies **StockCode-based overrides** for special cases (`'D'` → *Discount*, `'M'` → *Manual Adjustment*).

Additionally, operational anomalies (e.g., damages, missing items, wrong barcodes) are grouped as *Operational Adjustments*. For transactions marked as *Customer Returns*, the function further refines classifications into *Discount Reversal* or *Shipping Refund* where applicable.

This categorization helps distinguish between normal product sales and **non-standard adjustments/fees**, improving downstream analysis of sales, profitability, and operational issues.


In [ ]:
def categorize_item_types(df):
    # Default all to 'Product'
    df['Item_Type'] = 'Product'
    df['Description'] = df['Description'].astype(str).str.strip().str.lower()

    # Define keyword lists
    service_keywords = ['postage', 'dotcom postage']
    discount_keywords = ['discount']
    manual_keywords = ['manual']
    amazon_keywords = [
        'amazon', 'amazon fee', 'amazon sales', 'amazon sold sets',
        'sold as set on dotcom and amazon', 'amazon adjustment', 'amazon adjust'
    ]
    operational_keywords = [
        'damages', 'samples/damages', 'damages/display', 'damages?', 'damages/showroom etc',
        'damages/credits from asos.', 'damages/dotcom?', 'damages/samples', 'damages wax',
        '????damages????', 'possible damages or lost?', 'incorrectly made-thrown away.',
        'wrong barcode', 'wrong barcode (22467)', 'sold with wrong barcode',
        'wrongly sold as sets', 'wrongly sold sets', 'wrongly sold (22719) barcode',
        'stock check', 'check?', '??', '???', '???lost', '?? missing',
        '????missing', '???missing', 'lost??',
        'wet damaged', 'reverse 21/5/10 adjustment', 'adjustment', 'adjust',
        'reverse previous adjustment', 'adjust bad debt', 'taig adjust', 'taig adjust no stock',
        'temp adjustment', 'oops ! adjustment', 'dotcom adjust', 'missing', '?missing',
        'missing?', 'faulty', 're-adjustment'
    ]

    # StockCode rules
    stockcode_rules = {
        'd': 'Discount',
        'm': 'Manual Adjustment',
        'post': 'Shipping/Service',
        'amazon': 'Amazon Related'
    }
    for code, item_type in stockcode_rules.items():
        mask = df['StockCode'].astype(str).str.lower() == code.lower()
        df.loc[mask, 'Item_Type'] = item_type

    # Description-based rules
    description_rules = {
        'Shipping/Service': service_keywords,
        'Discount': discount_keywords,
        'Manual Adjustment': manual_keywords,
        'Amazon Related': amazon_keywords
    }
    for item_type, keywords in description_rules.items():
        pattern = r'\b(?:' + '|'.join(re.escape(k) for k in keywords) + r')\b'
        mask = df['Description'].str.contains(pattern, na=False, regex=True)
        df.loc[mask & (df['Item_Type'] == 'Product'), 'Item_Type'] = item_type

    # Operational keywords with word boundaries to avoid false positives like "pink check"
    operational_pattern = r'\b(?:' + '|'.join(re.escape(k) for k in operational_keywords) + r')\b'
    operational_mask = df['Description'].str.contains(operational_pattern, na=False, regex=True)
    df.loc[operational_mask & (df['Item_Type'] == 'Product'), 'Item_Type'] = 'Operational Adjustment'

    # Special handling for returns
    return_mask = df['Transaction_Type'] == 'Customer Return'
    df.loc[return_mask & (df['Item_Type'] == 'Discount'), 'Item_Type'] = 'Discount Reversal'
    df.loc[return_mask & (df['Item_Type'] == 'Shipping/Service'), 'Item_Type'] = 'Shipping Refund'

    # Summary
    print("Item type distribution:")
    print(df['Item_Type'].value_counts().to_string())

    return df
dff =  categorize_item_types(dff)

## Handle Zero and Anomalous Prices

`handle_zero_and_anomalous_prices` cleans the `Price` column by converting values to numeric, dropping invalid entries, and removing *Product* rows with zero or negative prices. This ensures only valid prices remain for accurate sales analysis.


In [ ]:
def handle_zero_and_anomalous_prices(df):
    # Convert Price to numeric
    df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

    # Drop rows with invalid (non-numeric) Price
    df.dropna(subset=['Price'], inplace=True)

    # Identify 'Product' rows with non-positive Price
    mask_invalid_price = (df['Item_Type'] == 'Product') & (df['Price'] <= 0)
    deleted_rows = df[mask_invalid_price].copy()

    # Remove them from the main DataFrame
    df = df[~mask_invalid_price].copy()

    print(f"Removed {len(deleted_rows)} 'Product' rows with non-positive prices.")

    return df

dff = handle_zero_and_anomalous_prices(dff)

## Extract Time Features

`extract_time_features` converts `InvoiceDate` to datetime, drops invalid entries, and derives new time-based features (`Year`, `Month`, `Day`, `DayOfWeek`, `Hour`). These features enable temporal analysis of sales trends and customer behavior.

In [ ]:
def extract_time_features(df):
    # Convert 'InvoiceDate' to datetime
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
    initial_rows = len(df)

    # Drop rows where InvoiceDate conversion failed
    df.dropna(subset=['InvoiceDate'], inplace=True)
    dropped = initial_rows - len(df)

    if dropped > 0:
        print(f"Dropped {dropped} rows due to invalid 'InvoiceDate'.")

    # Create time-based features
    df['InvoiceYear'] = df['InvoiceDate'].dt.year
    df['InvoiceMonth'] = df['InvoiceDate'].dt.month
    df['InvoiceDay'] = df['InvoiceDate'].dt.day
    df['InvoiceDayOfWeek'] = df['InvoiceDate'].dt.dayofweek  # Monday=0, Sunday=6
    df['InvoiceHour'] = df['InvoiceDate'].dt.hour

    print("Time-based features (Year, Month, Day, DayOfWeek, Hour) extracted.")
    return df
dff = extract_time_features(dff)
dff.head()

## Sales Summary

`sales_summary` generates a post-cleaning overview of sales data. It reports key metrics such as total transactions, invoices, customers, date range, average items per invoice, gross/net revenue, and unique products. It also highlights the top 5 countries by sales and shows distributions of transaction and item types for quick business insights.

In [ ]:
## Creating Total Price column
dff.loc[:, 'Total_Price'] = dff['Quantity'] * dff['Price']
print("'Total_Price' column calculated.")

In [ ]:
def sales_summary(df):
    # Filter for actual product sales
    sales_df = df[(df['Transaction_Type'] == 'Sale') & (df['Item_Type'] == 'Product')]
    returns_df = df[(df['Transaction_Type'] == 'Customer Return') & (df['Item_Type'] == 'Product')]

    # Core metrics
    total_transactions = len(df)
    unique_invoices = df['Invoice'].nunique()
    unique_customers = df['Customer ID'].nunique()
    date_range_start = df['InvoiceDate'].min()
    date_range_end = df['InvoiceDate'].max()
    average_items_per_invoice = df.groupby('Invoice')['Quantity'].sum().mean()

    # Revenue calculations
    total_gross_revenue = sales_df['Total_Price'].sum()
    total_net_revenue = total_gross_revenue + returns_df['Total_Price'].sum()

    # Other insights
    top_countries_by_sales = sales_df.groupby('Country')['Total_Price'].sum().nlargest(5)
    unique_products = sales_df['StockCode'].nunique()

    # Output
    print(f"Total transactions: {total_transactions}")
    print(f"Unique invoices: {unique_invoices}")
    print(f"Unique customers: {unique_customers}")
    print(f"Date range: {date_range_start.date()} to {date_range_end.date()}")
    print(f"Average items per invoice: {average_items_per_invoice:.2f}")
    print(f"Total gross revenue (products only): Rs. {total_gross_revenue:,.2f}")
    print(f"Total net revenue (sales - returns): Rs. {total_net_revenue:,.2f}")
    print(f"Number of unique products sold: {unique_products}")

    print("\nTop 5 countries by product sales:")
    print(top_countries_by_sales)

    print("\nTransaction_Type distribution:")
    print(df['Transaction_Type'].value_counts())

    print("\nItem_Type distribution:")
    print(df['Item_Type'].value_counts())
sales_summary(dff)


In [ ]:
dff.to_csv('cleaned_data.csv', index=False)
data = dff.copy()

# Task 3

Split the dataset by country and focus on a selected country (e.g., India) for detailed analysis. Visualize the country-specific data, analyze product prices, and categorize them into ranges (low, medium, high). Separate transactions into sales and returns while also analyzing sales across price segments.



## Country-wise Data Split

This code groups the dataset by `Country`, creating a dictionary of DataFrames for each. It prints the number of transactions per country, then builds a summary showing row/column counts, average rows, and identifies the countries with the most and fewest transactions.

In [ ]:
# Create dictionary of DataFrames by country
dfs_by_country = {country: data for country, data in data.groupby('Country')}

# Print shapes for all countries
print("\nNumber of transactions by country:")
print("-" * 40)
for country, df in dfs_by_country.items():
    print(f"{country.ljust(20)}: {len(df):,} rows")

# Summary statistics
country_stats = pd.DataFrame({
    'Country': dfs_by_country.keys(),
    'Rows': [df.shape[0] for df in dfs_by_country.values()],
    'Columns': [df.shape[1] for df in dfs_by_country.values()]
})

print("\nSummary Statistics:")
print("-" * 40)
print(f"Total countries: {len(dfs_by_country)}")
print(f"Average rows per country: {country_stats['Rows'].mean():,.0f}")
print(f"Country with most transactions: {country_stats.loc[country_stats['Rows'].idxmax(), 'Country']}")
print(f"Country with fewest transactions: {country_stats.loc[country_stats['Rows'].idxmin(), 'Country']}")

## Visualize Country Data - INDIA

`visualize_country_data` generates a dashboard of plots for a given country, showing transaction and item type distributions, top products (by quantity and revenue), top customers, monthly sales trends, hourly and weekday sales patterns, and a correlation heatmap of key numeric fields. It provides a comprehensive visual overview of country-level sales behavior.


In [ ]:
# Access the DataFrame for 'India'
df_india = dfs_by_country['India']
df_india.head()

In [ ]:
def visualize_country_data(df_country, country_name):
    plt.style.use('seaborn-v0_8')
    fig, axes = plt.subplots(4, 2, figsize=(18, 20))
    fig.suptitle(f"Data Overview for {country_name}", fontsize=20, fontweight="bold")

    # Transaction Type Distribution
    df_country['Transaction_Type'].value_counts().plot(
        kind='pie', autopct='%1.1f%%', ax=axes[0,0], startangle=90, colors=sns.color_palette("pastel"))
    axes[0,0].set_title("Transaction Type Distribution")
    axes[0,0].set_ylabel("")

    # Item Type Distribution
    df_country['Item_Type'].value_counts().plot(
        kind='pie', autopct='%1.1f%%', ax=axes[0,1], startangle=90, colors=sns.color_palette("pastel"))
    axes[0,1].set_title("Item Type Distribution")
    axes[0,1].set_ylabel("")

    # Top Selling Products by Quantity
    top_products_quantity = df_country.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)
    top_products_quantity.plot(kind='bar', ax=axes[1,0], color='skyblue')
    axes[1,0].set_title("Top 10 Products by Quantity Sold")
    axes[1,0].set_ylabel("Quantity")
    axes[1,0].tick_params(axis='x', rotation=45)
    axes[1,0].set_xticklabels(axes[1,0].get_xticklabels(), ha='right')

    # Top Selling Products by Revenue
    top_products_revenue = df_country.groupby('Description')['Total_Price'].sum().sort_values(ascending=False).head(10)
    top_products_revenue.plot(kind='bar', ax=axes[1,1], color='salmon')
    axes[1,1].set_title("Top 10 Products by Revenue")
    axes[1,1].set_ylabel("Revenue")
    axes[1,1].tick_params(axis='x', rotation=45)
    axes[1,1].set_xticklabels(axes[1,1].get_xticklabels(), ha='right')

    # Top Customers by Spend
    top_customers = df_country.groupby('Customer ID')['Total_Price'].sum().sort_values(ascending=False).head(10)
    top_customers.plot(kind='bar', ax=axes[2,0], color='orange')
    axes[2,0].set_title("Top 10 Customers by Spend")
    axes[2,0].set_ylabel("Total Spend")
    axes[2,0].tick_params(axis='x', rotation=45)
    axes[2,0].set_xticklabels(axes[2,0].get_xticklabels(), ha='right')

    # Monthly Sales Trend
    sales_trend = df_country.set_index('InvoiceDate').resample('ME')['Total_Price'].sum()
    sales_trend.plot(ax=axes[2,1], marker='o', color='green')
    axes[2,1].set_title("Monthly Sales Trend")
    axes[2,1].set_ylabel("Total Spend")
    axes[2,1].tick_params(axis='x', rotation=45)

    # Sales by Hour of Day
    sales_by_hour = df_country.groupby('InvoiceHour')['Total_Price'].sum()
    sales_by_hour.plot(kind='bar', ax=axes[3,0], color='purple')
    axes[3,0].set_title("Sales by Hour of Day")
    axes[3,0].set_xlabel("Hour of Day")
    axes[3,0].set_ylabel("Total Spend")
    axes[3,0].tick_params(axis='x', rotation=0)

    # Sales by Day of Week
    sales_by_dayofweek = df_country.groupby('InvoiceDayOfWeek')['Total_Price'].sum()
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    sales_by_dayofweek.index = sales_by_dayofweek.index.map(lambda x: days[x])
    sales_by_dayofweek.plot(kind='bar', ax=axes[3,1], color='brown')
    axes[3,1].set_title("Sales by Day of Week")
    axes[3,1].set_xlabel("Day of Week")
    axes[3,1].set_ylabel("Total Spend")
    axes[3,1].tick_params(axis='x', rotation=45)
    axes[3,1].set_xticklabels(axes[3,1].get_xticklabels(), ha='right')

    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    plt.show()

visualize_country_data(df_india, "India")

## Analyze Prices

`analyze_prices` generates side-by-side visualizations of a numeric column (default `Total_Price`), using a boxplot to highlight outliers and spread, and a violin plot to show the underlying distribution and density. It provides a quick yet comprehensive view of data dispersion, skewness, and anomalies, helping identify unusual price patterns.

In [ ]:
def analyze_prices(df, col='Total_Price'):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # Boxplot (IQR and outliers)
    sns.boxplot(x=df[col], ax=ax1, color='skyblue')
    ax1.set_title(f'Boxplot of {col}')

    # Violinplot (distribution density)
    sns.violinplot(x=df[col], ax=ax2, color='lightgreen')
    ax2.set_title(f'Distribution of {col}')

    plt.show()  # Prevents auto-display
# 1. Run analysis (no plots shown yet)
analyze_prices(df_india)

## Categorize Price Ranges

`categorize_price_ranges` bins a numeric column (default `Total_Price`) into predefined intervals such as `< -1000`, `-1000 to -100`, `-100 to 0`, `0 to 100`, `100 to 1000`, and `> 1000`. It adds a new categorical column (`Price_Range`) to the DataFrame for easier segmentation, and produces a summary table of counts per range. This dual output allows both **row-level categorization** and **aggregated insights** into how records are distributed across price bands, making it easier to detect trends, outliers, and concentration zones in the data.

In [ ]:
def categorize_price_ranges(df, price_col="Total_Price", country="India"):
    # Make a copy to avoid modifying original
    df_copy = df.copy()

    # Define price ranges
    price_bins = [-float('inf'), -1000, -100, 0, 100, 1000, float('inf')]
    price_labels = ['< -1000', '-1000 to -100', '-100 to 0',
                    '0 to 100', '100 to 1000', '> 1000']

    # Categorize into bins
    df_copy['Price_Range'] = pd.cut(
        df_copy[price_col],
        bins=price_bins,
        labels=price_labels,
        right=False  # left-inclusive, right-exclusive
    )

    # Count distribution
    price_range_counts = (
        df_copy[price_col]
        .groupby(pd.cut(df_copy[price_col],
                        bins=price_bins,
                        labels=price_labels,
                        right=False),
                 observed=False)
        .size()
        .reset_index(name="Count")
        .rename(columns={price_col: "Price Range"})
    )

    print(f"Distribution of {price_col} for {country} by Range:")
    display(price_range_counts)

    print(f"\nDataFrame with Price_Range column for {country}:")
    display(df_copy.head())

    return df_copy
df_india = categorize_price_ranges(df_india)

## Separate Sales, Returns, and Price Segments

`separate_sales_and_returns` partitions the dataset into **sales** and **returns**, further isolating transactions involving only `Product` items. It returns four DataFrames: product-level sales, all sales, product-level returns, and all returns, while printing their shapes for verification.

When combined with **price segmentation**, the function enables focused analysis within specific price brackets (e.g., `< -1000`, `-1000 to -100`, `-100 to 0`, `0 to 100`, `100 to 1000`, `> 1000`). Analysts can zoom into subsets like low-value (`0–100`) or extreme negative values (`< -1000`) to compare **sales vs. returns patterns**, **anomalies**, and **profitability** across ranges.

In [ ]:
def separate_sales_and_returns(df):
    # Separate sales and returns
    sales_df = df[df['Transaction_Type'] == 'Sale'].copy()
    returns_df = df[df['Transaction_Type'] == 'Customer Return'].copy()

    # Filter only 'Product' items from sales
    sales_df_products = sales_df[sales_df['Item_Type'] == 'Product'].copy()
    returns_df_products = returns_df[returns_df['Item_Type'] == 'Product'].copy()


    # Print shapes for confirmation
    print(f"\nSales transactions (products only) shape: {sales_df_products.shape}")
    print(f"Sales transactions (all) shape: {sales_df.shape}")
    print(f"Return transactions (products only) shape: {returns_df_products.shape}")
    print(f"Return transactions (all) shape: {returns_df.shape}")

    return sales_df_products, sales_df, returns_df_products, returns_df

In [ ]:
# price_labels = ['< -1000', '-1000 to -100', '-100 to 0', '0 to 100', '100 to 1000', '> 1000']
df_less_than_minus_1000 = df_india[df_india['Price_Range'] == '< -1000']
df_minus_1000_to_minus_100 = df_india[df_india['Price_Range'] == '-1000 to -100']
df_minus_100_to_0 = df_india[df_india['Price_Range'] == '-100 to 0']
df_0_to_100 = df_india[df_india['Price_Range'] == '0 to 100']
df_100_to_1000 = df_india[df_india['Price_Range'] == '100 to 1000']
df_greater_than_1000 = df_india[df_india['Price_Range'] == '> 1000']

In [ ]:
sales_df_products, sales_df, returns_df_products, returns_df = separate_sales_and_returns(df_0_to_100)

In [ ]:
analyze_prices(sales_df_products)

# Task 4

Perform advanced analysis by conducting RFM (Recency, Frequency, Monetary) analysis for customers. Carry out cohort analysis to study customer retention over time. Analyze overall sales performance and trends across different time periods. Evaluate the impact of returns and refunds on both sales and customer value.



## RFM Analysis

`rfm` computes **Recency, Frequency, and Monetary (RFM) scores** for each customer to evaluate their purchase behavior.

* **Recency** measures how recently a customer purchased.
* **Frequency** captures how often they purchased.
* **Monetary** reflects the total spend.

The function assigns **R, F, and M scores (1–5)** using quintiles, then combines them into an **RFM code** and segments customers into categories such as *Champions*, *Loyal Customers*, *Potential Loyalists*, *At Risk*, and *Lost*.

It returns:

1. **`rfm_df`** → detailed RFM scores and assigned segment for each customer.
2. **`rfm_segment_profiles`** → aggregated segment profiles showing average metrics, number of customers, and contribution to total revenue (both absolute and percentage).

This analysis provides a structured way to identify high-value customers, retention risks, and growth opportunities, making it a cornerstone for customer segmentation and targeted marketing strategies.

In [ ]:
def rfm(df):
    # Ensure datetime format
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

    # Snapshot date = 1 day after the last purchase
    snapshot_date = df['InvoiceDate'].max() + timedelta(days=1)

    # Calculate Recency, Frequency, Monetary
    rfm_df = df.groupby('Customer ID').agg(
        Recency=('InvoiceDate', lambda x: (snapshot_date - x.max()).days),
        Frequency=('Invoice', 'nunique'),
        Monetary=('Total_Price', 'sum')
    ).reset_index()

    # Filter out invalid customers
    rfm_df = rfm_df[(rfm_df['Frequency'] > 0) & (rfm_df['Monetary'] > 0)]

    # Scoring (5 = best for Recency, 5 = best for Frequency/Monetary)
    rfm_df['R_Score'] = pd.qcut(rfm_df['Recency'].rank(method='first'), 5, labels=[5,4,3,2,1])
    rfm_df['F_Score'] = pd.qcut(rfm_df['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
    rfm_df['M_Score'] = pd.qcut(rfm_df['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5])

    # Combine into single RFM code
    rfm_df['RFM_Score'] = rfm_df['R_Score'].astype(str) + \
                          rfm_df['F_Score'].astype(str) + \
                          rfm_df['M_Score'].astype(str)

    # Segment customers
    def rfm_segment(row):
        r, f, m = int(row['R_Score']), int(row['F_Score']), int(row['M_Score'])
        if r == 5 and f == 5 and m == 5:
            return 'Champions'
        elif f >= 4 and m >= 4:
            return 'Loyal Customers'
        elif r >= 4 and f >= 3:
            return 'Potential Loyalists'
        elif r == 5 and f == 1:
            return 'New Customers'
        elif r <= 2 and f >= 3:
            return 'At Risk'
        elif r <= 2 and f <= 2:
            return 'Lost'
        else:
            return 'Others'

    rfm_df['Segment'] = rfm_df.apply(rfm_segment, axis=1)


    # Aggregate segment profiles
    rfm_segment_profiles = rfm_df.groupby('Segment').agg(
        Avg_Recency=('Recency', 'mean'),
        Avg_Frequency=('Frequency', 'mean'),
        Avg_Monetary=('Monetary', 'mean'),
        Num_Customers=('Customer ID', 'count')
    ).reset_index()

    # Add percentages
    rfm_segment_profiles['Percentage_of_Customers'] = (
        rfm_segment_profiles['Num_Customers'] / rfm_df.shape[0] * 100
    ).round(2)

    total_revenue = rfm_df['Monetary'].sum()
    rfm_segment_profiles['Percentage_of_Revenue'] = (
        rfm_segment_profiles['Avg_Monetary'] * rfm_segment_profiles['Num_Customers'] / total_revenue * 100
    ).round(2)


    return rfm_df, rfm_segment_profiles

In [ ]:
rfm_table, rfm_profiles = rfm(sales_df_products)

# Print preview
display("Sample RFM data with segments:")
display(rfm_table.head())

# Print profiles
display("RFM Segment Profiles:")
display(rfm_profiles.sort_values(by='Percentage_of_Revenue', ascending=False))

In [ ]:
def plot_rfm_profiles(rfm_profiles):
  # Sort profiles by Percentage_of_Revenue for better visualization
  rfm_profiles_sorted = rfm_profiles.sort_values(by='Percentage_of_Revenue', ascending=False)

  fig, axes = plt.subplots(2, 2, figsize=(18, 12))
  fig.suptitle('RFM Segment Profiles Visualization', fontsize=16)

  # Plot 1: Percentage of Customers by Segment
  sns.barplot(x='Segment', y='Percentage_of_Customers', data=rfm_profiles_sorted, ax=axes[0, 0])
  axes[0, 0].set_title('Percentage of Customers by RFM Segment')
  axes[0, 0].set_ylabel('Percentage (%)')
  axes[0, 0].tick_params(axis='x', rotation=45)
  plt.setp(axes[0, 0].get_xticklabels(), ha='right')

  # Plot 2: Percentage of Revenue by Segment
  sns.barplot(x='Segment', y='Percentage_of_Revenue', data=rfm_profiles_sorted, ax=axes[0, 1])
  axes[0, 1].set_title('Percentage of Revenue by RFM Segment')
  axes[0, 1].set_ylabel('Percentage (%)')
  axes[0, 1].tick_params(axis='x', rotation=45)
  plt.setp(axes[0, 1].get_xticklabels(), ha='right')

  # Plot 3: Average Monetary Value by Segment
  sns.barplot(x='Segment', y='Avg_Monetary', data=rfm_profiles_sorted, ax=axes[1, 0])
  axes[1, 0].set_title('Average Monetary Value by RFM Segment')
  axes[1, 0].set_ylabel('Average Monetary Value')
  axes[1, 0].tick_params(axis='x', rotation=45)
  plt.setp(axes[1, 0].get_xticklabels(), ha='right')

  # Plot 4: Average Frequency by Segment
  sns.barplot(x='Segment', y='Avg_Frequency', data=rfm_profiles_sorted, ax=axes[1, 1])
  axes[1, 1].set_title('Average Frequency by RFM Segment')
  axes[1, 1].set_ylabel('Average Frequency')
  axes[1, 1].tick_params(axis='x', rotation=45)
  plt.setp(axes[1, 1].get_xticklabels(), ha='right')

  plt.tight_layout(rect=[0, 0.03, 1, 0.95])
  plt.show()

plot_rfm_profiles(rfm_profiles)

## Cohort Analysis

`cohort_analysis` performs customer cohort analysis by grouping customers based on their first purchase month and tracking their retention and spending behavior over time. It generates two heatmaps: one showing retention rates (% of customers who return in subsequent months) and another showing average revenue per active customer within each cohort. This helps businesses measure customer loyalty, monitor churn patterns, and understand the long-term value of different cohorts.


In [ ]:
def cohort_analysis(df, date_col='InvoiceDate', customer_col='Customer ID', value_col='Total_Price'):
    cohort_df = df.copy()
    cohort_df[date_col] = pd.to_datetime(cohort_df[date_col])

    # Extract Invoice Month
    cohort_df['InvoiceMonth'] = cohort_df[date_col].dt.to_period('M')

    # First purchase month for each customer
    customer_first_purchase = cohort_df.groupby(customer_col)['InvoiceMonth'].min().reset_index()
    customer_first_purchase.columns = [customer_col, 'CohortMonth']

    # Merge cohort month into main DF
    cohort_df = pd.merge(cohort_df, customer_first_purchase, on=customer_col)

    # Cohort index
    def get_cohort_index(row):
        year_diff = row['InvoiceMonth'].year - row['CohortMonth'].year
        month_diff = row['InvoiceMonth'].month - row['CohortMonth'].month
        return year_diff * 12 + month_diff

    cohort_df['CohortIndex'] = cohort_df.apply(get_cohort_index, axis=1)

    # Retention matrix
    cohort_counts = cohort_df.groupby(['CohortMonth', 'CohortIndex'])[customer_col].nunique().reset_index()
    cohort_counts.rename(columns={customer_col: 'TotalCustomers'}, inplace=True)
    cohort_pivot = cohort_counts.pivot(index='CohortMonth', columns='CohortIndex', values='TotalCustomers')
    cohort_sizes = cohort_pivot.iloc[:, 0]
    retention_matrix = cohort_pivot.divide(cohort_sizes, axis=0) * 100

    # Revenue per active customer
    cohort_revenue = cohort_df.groupby(['CohortMonth', 'CohortIndex'])[value_col].sum().reset_index()
    cohort_avg_revenue = cohort_revenue.pivot(index='CohortMonth', columns='CohortIndex', values=value_col)
    cohort_avg_revenue = cohort_avg_revenue.divide(cohort_pivot, axis=0)

    # --- Combined subplot figure ---
    fig, axes = plt.subplots(1, 2, figsize=(26, 10))

    # Plot 1: Retention
    sns.heatmap(retention_matrix, annot=True, fmt='.1f', cmap='YlGnBu', linewidths=.5,
                cbar_kws={'label': 'Retention Rate (%)'}, ax=axes[0])
    axes[0].set_title('Cohort Retention Analysis')
    axes[0].set_ylabel('Cohort Month (First Purchase)')
    axes[0].set_xlabel('Months Since First Purchase')

    # Plot 2: Revenue per active customer
    sns.heatmap(cohort_avg_revenue, annot=True, fmt='.1f', cmap='Blues', linewidths=.5,
                cbar_kws={'label': 'Average Revenue'}, ax=axes[1])
    axes[1].set_title('Cohort Average Revenue per Active Customer')
    axes[1].set_ylabel('Cohort Month (First Purchase)')
    axes[1].set_xlabel('Months Since First Purchase')

    plt.tight_layout()
    plt.show()

    return retention_matrix, cohort_avg_revenue


In [ ]:
retention_matrix, avg_rev_matrix = cohort_analysis(sales_df_products)

## Sales Performance & Trend Analysis

`sales_performance_analysis` analyzes overall sales patterns by tracking both daily and monthly revenue trends, helping to identify seasonality and growth. It also highlights the top-performing products in terms of total revenue and quantity sold. By combining time-based trends with product-level insights, this analysis provides a clear view of sales dynamics, customer demand, and key revenue drivers.

In [ ]:
def analyze_sales_trends(sales_df_products):
    # Daily and Monthly Sales Trends
    daily_sales = sales_df_products.set_index('InvoiceDate').resample('D')['Total_Price'].sum().fillna(0)
    monthly_sales = sales_df_products.set_index('InvoiceDate').resample('ME')['Total_Price'].sum().fillna(0)

    # Create subplots for sales trends (side by side)
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('Sales Revenue Trends Over Time', fontsize=16)

    # Plot Daily Sales Trend
    axes[0].plot(daily_sales.index, daily_sales.values, label='Daily Sales Revenue')
    axes[0].set_title('Daily Sales Revenue Over Time')
    axes[0].set_xlabel('Date')
    axes[0].set_ylabel('Revenue')
    axes[0].grid(True)
    axes[0].legend()

    # Plot Monthly Sales Trend
    axes[1].plot(monthly_sales.index, monthly_sales.values, label='Monthly Sales Revenue', marker='o')
    axes[1].set_title('Monthly Sales Revenue Over Time (Seasonality)')
    axes[1].set_xlabel('Month')
    axes[1].set_ylabel('Total_Price')
    axes[1].grid(True)
    axes[1].legend()

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

    # Product Performance
    product_performance = sales_df_products.groupby('StockCode').agg(
        Total_Quantity_Sold=('Quantity', 'sum'),
        Total_Revenue=('Total_Price', 'sum')
    ).reset_index()

    top_products_revenue = product_performance.sort_values(by='Total_Revenue', ascending=False).head()
    top_products_quantity = product_performance.sort_values(by='Total_Quantity_Sold', ascending=False).head()

    print("\nTop 5 Products by Revenue:")
    display(top_products_revenue)

    print("\nTop 5 Products by Quantity Sold:")
    display(top_products_quantity)

    return top_products_revenue, top_products_quantity

In [ ]:
top_revenue, top_quantity = analyze_sales_trends(sales_df_products)

## Returns & Refunds Impact Analysis

`returns_refunds_analysis` evaluates the business impact of product returns by quantifying both the total quantity of items returned and the total refund value. It tracks monthly refund trends to reveal fluctuations and patterns in returns over time. Additionally, it highlights the top products driving returns, both by quantity and by refund value, enabling businesses to identify problematic products or categories. This analysis provides insights into return-related revenue loss and helps in improving product quality, customer satisfaction, and inventory management.

In [ ]:
def returns_refunds_analysis(returns_df, date_col='InvoiceDate', quantity_col='Quantity', value_col='Total_Price', product_col='StockCode'):
    # Ensure datetime
    returns_df[date_col] = pd.to_datetime(returns_df[date_col])

    # Summary
    total_returns_quantity = returns_df[quantity_col].sum()
    total_refund_value = returns_df[value_col].sum()

    print("\n--- Returns and Refunds Impact Assessment ---")
    print(f"Total quantity of items returned: {-total_returns_quantity}")
    print(f"Total monetary value of refunds: {total_refund_value}")

    #  Monthly return trends
    returns_monthly = returns_df.set_index(date_col).resample('ME')[[value_col]].sum().fillna(0)

    plt.figure(figsize=(10, 5))
    plt.plot(returns_monthly.index, returns_monthly[value_col], label='Monthly Refund Value', marker='o', color='red')
    plt.title('Monthly Refund Value Over Time')
    plt.xlabel('Month')
    plt.ylabel('Refund Value')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Top returned products
    top_returned_products = returns_df.groupby(product_col).agg(
        Total_Quantity_Returned=(quantity_col, 'sum'),
        Total_Refund_Value=(value_col, 'sum')
    ).reset_index()

    # Absolute values for ranking
    top_returned_products['Abs_Quantity_Returned'] = top_returned_products['Total_Quantity_Returned'].abs()
    top_returned_products['Abs_Refund_Value'] = top_returned_products['Total_Refund_Value'].abs()

    top_5_returned_by_quantity = top_returned_products.sort_values(by='Abs_Quantity_Returned', ascending=False).head(5)
    top_5_returned_by_value = top_returned_products.sort_values(by='Abs_Refund_Value', ascending=False).head(5)

    print("\nTop 5 Products by Quantity Returned:")
    display(top_5_returned_by_quantity[[product_col, 'Total_Quantity_Returned', 'Total_Refund_Value']])

    print("\nTop 5 Products by Refund Value:")
    display(top_5_returned_by_value[[product_col, 'Total_Quantity_Returned', 'Total_Refund_Value']])

    return {
        "total_quantity_returned": -total_returns_quantity,
        "total_refund_value": total_refund_value,
        "monthly_trends": returns_monthly
    }

In [ ]:
returns_refunds_analysis(returns_df)

# Task 5

Prepare the dataset for clustering by performing advanced customer segmentation. Create a combined dataset that merges RFM metrics with additional segmentation features. Build clustering-ready datasets using multiple approaches such as RFM, Value–Volume, and Temporal Behavioral models. Apply feature scaling to standardize the data for clustering algorithms.

## Advanced Customer Segmentation

`advanced_customer_segmentation` builds enriched customer profiles by calculating detailed behavioral and transactional metrics. It goes beyond standard RFM by adding features such as **average order value, total items purchased, unique products purchased, average items per order, and average days between purchases**. It also incorporates **return behavior** by computing each customer’s return rate based on refund values.

This enriched dataset provides a comprehensive view of customer activity, enabling more effective **clustering, segmentation, and predictive modeling**. Businesses can use it to identify loyal customers, high-return-risk groups, and opportunities for targeted retention or marketing strategies.



In [ ]:
def advanced_customer_segmentation(sales_df_products, returns_df):
    # Snapshot date for recency
    snapshot_date = sales_df_products['InvoiceDate'].max() + pd.Timedelta(days=1)

    # Aggregate customer-level metrics
    customer_level_df = sales_df_products.groupby('Customer ID').agg(
        Recency=('InvoiceDate', lambda date: (snapshot_date - date.max()).days),
        Frequency=('Invoice', lambda num: num.nunique()),
        Monetary=('Total_Price', 'sum'),
        Avg_Order_Value=('Total_Price', 'mean'),  # average revenue per transaction
        Total_Items_Purchased=('Quantity', 'sum'),
        Unique_Products_Purchased=('StockCode', 'nunique')
    ).reset_index()

    # Average items per order
    customer_level_df['Avg_Items_per_Order'] = (
        customer_level_df['Total_Items_Purchased'] / customer_level_df['Frequency']
    )

    # Average days between purchases
    customer_invoice_dates = sales_df_products.groupby('Customer ID')['InvoiceDate'].apply(list)
    avg_days_between = []
    for cid in customer_level_df['Customer ID']:
        dates = sorted(customer_invoice_dates[cid])
        if len(dates) > 1:
            diffs = np.diff(dates)
            avg_days_between.append(np.mean(diffs).days)
        else:
            avg_days_between.append(0)
    customer_level_df['Avg_Days_Between_Purchases'] = avg_days_between

    # Return rate (monetary)
    returns_summary = returns_df.groupby('Customer ID')['Total_Price'].sum().abs().reset_index()
    returns_summary.rename(columns={'Total_Price': 'Total_Return_Value'}, inplace=True)

    customer_level_df = customer_level_df.merge(returns_summary, on='Customer ID', how='left').fillna(0)
    customer_level_df['Return_Rate'] = customer_level_df['Total_Return_Value'] / customer_level_df['Monetary']
    customer_level_df['Return_Rate'] = customer_level_df['Return_Rate'].replace([np.inf, -np.inf], 0)

    return customer_level_df

In [ ]:
customer_level_df = advanced_customer_segmentation(sales_df_products, returns_df)
display(customer_level_df.head(1))
display(rfm_table.head(1))

## Combined RFM & Segmentation Dataset

`combine_rfm_and_segmentation` merges traditional RFM scoring with advanced customer-level segmentation features into a single dataset. This unified view allows analysts to leverage both **transaction recency/frequency/monetary behavior** and **extended engagement metrics** for deeper segmentation, clustering, and customer lifetime value analysis.

In [ ]:
def combine_rfm_and_segmentation(rfm_table, customer_level_df):
    # Merge on Customer ID
    combined_df = pd.merge(rfm_table, customer_level_df, on="Customer ID", how="inner")
    return combined_df

combined_df = combine_rfm_and_segmentation(rfm_table, customer_level_df)
combined_df.head(1)

In [ ]:
combined_df.columns

## Clustering Dataset Preparation

`prepare_clustering_df` extracts and organizes the most relevant features from the enriched customer dataset into a **clean, analysis-ready dataframe** for clustering tasks.

It ensures a consistent feature set by keeping both **core RFM scores** and **extended engagement metrics** (e.g., order behavior, product diversity, return patterns). This streamlined dataset eliminates noise and aligns customer records for direct use in **unsupervised learning models** like KMeans, GMM, or Hierarchical Clustering.

By standardizing the input, this function acts as a **final preprocessing step** before scaling, dimensionality reduction, and clustering, ensuring comparability across customers.


In [ ]:
def prepare_clustering_df(df):
    # Columns to keep
    keep_cols = [
        "Customer ID",
        "Recency_x", "Frequency_x", "Monetary_x", "R_Score", "F_Score", "M_Score",
        "Avg_Order_Value", "Total_Items_Purchased", "Unique_Products_Purchased",
        "Avg_Items_per_Order", "Avg_Days_Between_Purchases",
        "Total_Return_Value", "Return_Rate"
    ]

    clustering_df = df[keep_cols].copy()

    # Rename columns
    clustering_df.rename(columns={
        "Recency_x": "Recency",
        "Frequency_x": "Frequency",
        "Monetary_x": "Monetary"
    }, inplace=True)

    # Convert category/object scores to numeric
    for score_col in ["R_Score", "F_Score", "M_Score"]:
        clustering_df[score_col] = pd.to_numeric(clustering_df[score_col], errors="coerce")

    # Fill NaNs with 0
    clustering_df = clustering_df.fillna(0)

    # Convert everything except Customer ID to float
    for col in clustering_df.columns:
        if col != "Customer ID":
            clustering_df[col] = clustering_df[col].astype(float)

    return clustering_df

In [ ]:
clustering_df = prepare_clustering_df(combined_df)
display(clustering_df.head(1))

## Multi-Dimensional Approaches: RFM, Value–Volume, and Temporal Behavioral Model
The customer segmentation framework combines three perspectives:

* **RFM (g\_rfm):** measures recency, frequency, and monetary value to assess engagement and overall customer worth.

* **Value & Volume (g\_pvv):** adds monetary, average order value, total items, and items per order to identify high spenders, bulk buyers, and low-value groups for pricing and loyalty strategies.

* **Timing & Habit (g\_pth):** uses recency, frequency, and average days between purchases to distinguish habitual, occasional, and at-risk buyers, supporting retention and churn prediction.


In [ ]:
g_rfm = clustering_df[['Recency', 'Frequency', 'Monetary']].copy() #Customer Segmentation
g_pvv = clustering_df[['Monetary', 'Avg_Order_Value', 'Total_Items_Purchased', 'Avg_Items_per_Order']].copy() #Purchase Value Volume
g_pth = clustering_df[['Recency', 'Frequency', 'Avg_Days_Between_Purchases']].copy() #Purchase Timing & Habit

In [ ]:
def drop_cluster_columns(df):
    cluster_cols = [col for col in df.columns if col.endswith('_Cluster')]
    df.drop(columns=cluster_cols, inplace=True, errors='ignore')
    return df

In [ ]:
clustering_df = drop_cluster_columns(clustering_df)
g_rfm = drop_cluster_columns(g_rfm)
g_pvv = drop_cluster_columns(g_pvv)
g_pth = drop_cluster_columns(g_pth)

In [ ]:
def visualize_rfm_data(g_rfm, g_pvv, g_pth):
  # Set the style for the plots
  sns.set_theme(style="whitegrid")

  #  Visualizations for g_rfm (Recency, Frequency, Monetary)
  fig, axes = plt.subplots(1, 3, figsize=(18, 5))
  fig.suptitle('Distribution of RFM Features', fontsize=16)

  sns.histplot(g_rfm['Recency'], kde=True, ax=axes[0])
  axes[0].set_title('Distribution of Recency')
  axes[0].set_xlabel('Recency (Days)')
  axes[0].set_ylabel('Frequency')

  sns.histplot(g_rfm['Frequency'], kde=True, ax=axes[1])
  axes[1].set_title('Distribution of Frequency')
  axes[1].set_xlabel('Frequency')
  axes[1].set_ylabel('Frequency')
  axes[1].set_xlim(0, g_rfm['Frequency'].quantile(0.95)) # Limit x-axis for better visualization

  sns.histplot(g_rfm['Monetary'], kde=True, ax=axes[2])
  axes[2].set_title('Distribution of Monetary')
  axes[2].set_xlabel('Monetary Value')
  axes[2].set_ylabel('Frequency')
  axes[2].set_xlim(0, g_rfm['Monetary'].quantile(0.95))

  plt.tight_layout(rect=[0, 0.03, 1, 0.95])
  plt.show()


  # Visualizations for g_pvv (Purchase Value Volume)
  fig, axes = plt.subplots(2, 2, figsize=(16, 10))
  fig.suptitle('Distribution of Purchase Value/Volume Features', fontsize=16)

  sns.histplot(g_pvv['Monetary'], kde=True, ax=axes[0, 0])
  axes[0, 0].set_title('Distribution of Monetary Value')
  axes[0, 0].set_xlabel('Monetary Value')
  axes[0, 0].set_ylabel('Frequency')
  axes[0, 0].set_xlim(0, g_pvv['Monetary'].quantile(0.95))

  sns.histplot(g_pvv['Avg_Order_Value'], kde=True, ax=axes[0, 1])
  axes[0, 1].set_title('Distribution of Average Order Value')
  axes[0, 1].set_xlabel('Average Order Value')
  axes[0, 1].set_ylabel('Frequency')
  axes[0, 1].set_xlim(0, g_pvv['Avg_Order_Value'].quantile(0.95))


  sns.histplot(g_pvv['Total_Items_Purchased'], kde=True, ax=axes[1, 0])
  axes[1, 0].set_title('Distribution of Total Items Purchased')
  axes[1, 0].set_xlabel('Total Items Purchased')
  axes[1, 0].set_ylabel('Frequency')
  axes[1, 0].set_xlim(0, g_pvv['Total_Items_Purchased'].quantile(0.95))


  sns.histplot(g_pvv['Avg_Items_per_Order'], kde=True, ax=axes[1, 1])
  axes[1, 1].set_title('Distribution of Average Items per Order')
  axes[1, 1].set_xlabel('Average Items per Order')
  axes[1, 1].set_ylabel('Frequency')
  axes[1, 1].set_xlim(0, g_pvv['Avg_Items_per_Order'].quantile(0.95))


  plt.tight_layout(rect=[0, 0.03, 1, 0.95])
  plt.show()


  #  Visualizations for g_pth (Purchase Timing & Habit)
  fig, axes = plt.subplots(1, 3, figsize=(18, 5))
  fig.suptitle('Distribution of Purchase Timing & Habit Features', fontsize=16)

  sns.histplot(g_pth['Recency'], kde=True, ax=axes[0])
  axes[0].set_title('Distribution of Recency')
  axes[0].set_xlabel('Recency (Days)')
  axes[0].set_ylabel('Frequency')

  sns.histplot(g_pth['Frequency'], kde=True, ax=axes[1])
  axes[1].set_title('Distribution of Frequency')
  axes[1].set_xlabel('Frequency')
  axes[1].set_ylabel('Frequency')
  axes[1].set_xlim(0, g_pth['Frequency'].quantile(0.95))


  sns.histplot(g_pth['Avg_Days_Between_Purchases'], kde=True, ax=axes[2])
  axes[2].set_title('Distribution of Average Days Between Purchases')
  axes[2].set_xlabel('Average Days Between Purchases')
  axes[2].set_ylabel('Frequency')
  axes[2].set_xlim(0, g_pth['Avg_Days_Between_Purchases'].quantile(0.95))


  plt.tight_layout(rect=[0, 0.03, 1, 0.95])
  plt.show()

visualize_rfm_data(g_rfm, g_pvv, g_pth)

## Feature Scaling

`scaling` standardizes selected features with missing value handling, returning a scaled dataframe and the fitted scaler.



In [ ]:
features = [
    'Recency', 'Frequency', 'Monetary', 'R_Score', 'F_Score', 'M_Score', 'Avg_Order_Value',
    'Total_Items_Purchased', 'Unique_Products_Purchased',
    'Avg_Items_per_Order', 'Avg_Days_Between_Purchases', 'Return_Rate'
]
features_g_rfm = ['Recency', 'Frequency', 'Monetary']
features_g_pvv = ['Monetary', 'Avg_Order_Value', 'Total_Items_Purchased', 'Avg_Items_per_Order']
features_g_pth = ['Recency', 'Frequency', 'Avg_Days_Between_Purchases']

In [ ]:
def scaling(df, features, id_col="Customer ID"):
    X = df[features].copy()

    # Handle missing/infinite values
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    X.fillna(X.mean(), inplace=True)

    #  Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Return scaled dataframe with ID as index
    if id_col in df.columns:
        X_scaled_df = pd.DataFrame(X_scaled, columns=features, index=df[id_col])
    else:
        X_scaled_df = pd.DataFrame(X_scaled, columns=features)
    return X_scaled_df, scaler

In [ ]:
full_scaled, full_scaler = scaling(clustering_df, features) # Full feature set scaling
rfm_scaled, rfm_scaler = scaling(clustering_df, features_g_rfm) # RFM scaling
pvv_scaled, pvv_scaler = scaling(clustering_df, features_g_pvv) # Purchase Value & Volume scaling
pth_scaled, pth_scaler = scaling(clustering_df, features_g_pth) # Purchase Timing & Habit scaling

display(full_scaled.head(1))
display(rfm_scaled.head(1))
display(pvv_scaled.head(1))
display(pth_scaled.head(1))

# Task 6

Apply and compare multiple clustering algorithms including K-Means, Hierarchical, DBSCAN, Gaussian Mixture Models (GMM), Affinity Propagation, MeanShift, Spectral Clustering, HDBSCAN, OPTICS, and Birch. Assign cluster labels to customers and prepare results for evaluation.

## K-Means Clustering

`perform_kmeans_clustering` determines the optimal number of clusters using the **Elbow Method** and **Silhouette Score**, fits a final K-Means model, assigns cluster labels, and visualizes results through **SSE curves, silhouette plots, and heatmaps of cluster profiles**.


In [ ]:
def perform_kmeans_clustering(X_scaled, df, features, scaler):
    # Determine optimal K using Elbow Method and Silhouette Score
    sse = []
    silhouette_scores = []
    k_range = range(2, 11)  # Test K from 2 to 10

    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X_scaled)
        sse.append(kmeans.inertia_)
        score = silhouette_score(X_scaled, kmeans.labels_)
        silhouette_scores.append(score)

    # Find the elbow point using kneed
    knee_locator = KneeLocator(k_range, sse, curve='convex', direction='decreasing')
    optimal_k = knee_locator.knee
    print(f"Optimal K (Elbow Method): {optimal_k}")

    # Train final KMeans model with optimal_k
    kmeans_model = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
    kmeans_model.fit(X_scaled)  # Fit the model to assign labels

    # Add cluster labels to the original DataFrame
    df['KMeans_Cluster'] = kmeans_model.labels_

    print(f"K-Means Clustering with K={optimal_k} complete.")

    # Cluster profiles in original units (not scaled)
    cluster_profiles = pd.DataFrame(
        scaler.inverse_transform(
            pd.DataFrame(
                kmeans_model.cluster_centers_,
                columns=features
            )
        ),
        columns=features
    )
    cluster_profiles.index.name = 'Cluster'

    # Cluster profiles in scaled space (for heatmap)
    cluster_profiles_scaled = pd.DataFrame(kmeans_model.cluster_centers_, columns=features)
    cluster_profiles_scaled.index.name = 'Cluster'

    # Combined subplot figure
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Elbow Method
    axes[0].plot(k_range, sse, marker='o')
    axes[0].axvline(optimal_k, color='r', linestyle='--', label=f'Elbow = {optimal_k}')
    axes[0].set_title('Elbow Method for Optimal K')
    axes[0].set_xlabel('Number of Clusters (K)')
    axes[0].set_ylabel('SSE')
    axes[0].legend()

    # Silhouette Score
    axes[1].plot(k_range, silhouette_scores, marker='o')
    axes[1].set_title('Silhouette Score for K')
    axes[1].set_xlabel('Number of Clusters (K)')
    axes[1].set_ylabel('Silhouette Score')

    # Heatmap of scaled cluster profiles
    sns.heatmap(cluster_profiles_scaled.T, annot=True, cmap='viridis', fmt=".2f", ax=axes[2])
    axes[2].set_title('K-Means Cluster Profiles (Scaled Feature Means)')
    axes[2].set_xlabel('Cluster')
    axes[2].set_ylabel('Feature')

    plt.tight_layout()
    plt.show()

    return df, optimal_k, kmeans_model

In [ ]:
# Call the function to perform clustering
clustering_df, optimal_k, kmeans_model = perform_kmeans_clustering(full_scaled, clustering_df.copy(), features, full_scaler)

In [ ]:
# RFM clustering
g_rfm, optimal_k_rfm, kmeans_model_rfm = perform_kmeans_clustering(rfm_scaled, g_rfm.copy(), features_g_rfm, rfm_scaler)

In [ ]:
# Purchase Value & Volume clustering
g_pvv, optimal_k_pvv, kmeans_model_pvv = perform_kmeans_clustering(pvv_scaled, g_pvv.copy(), features_g_pvv, pvv_scaler)

In [ ]:
# Purchase Timing & Habit clustering
g_pth, optimal_k_pth, kmeans_model_pth = perform_kmeans_clustering(pth_scaled, g_pth.copy(), features_g_pth, pth_scaler)

In [ ]:
def visualize_curr_cluster(X_scaled, df, cluster_col):
    # --- Determine palette based on number of clusters ---
    n_clusters = df[cluster_col].nunique()
    if n_clusters <= 5:
        palette = 'viridis'   # continuous
        legend_type = 'colorbar'
    elif n_clusters <= 10:
        palette = 'tab10'     # categorical
        legend_type = 'full'
    elif n_clusters <= 20:
        palette = 'tab20'     # categorical
        legend_type = 'full'
    else:
        palette = sns.color_palette("hsv", n_colors=n_clusters)  # categorical
        legend_type = 'full'

    # --- PCA ---
    pca = PCA(n_components=2, random_state=42)
    X_pca = pca.fit_transform(X_scaled)
    pca_df = pd.DataFrame(X_pca, columns=['PCA1', 'PCA2'])
    pca_df[cluster_col] = df[cluster_col].values

    # --- t-SNE ---
    tsne = TSNE(n_components=2, perplexity=30, random_state=42, max_iter=1000)
    X_tsne = tsne.fit_transform(X_scaled)
    tsne_df = pd.DataFrame(X_tsne, columns=['TSNE1', 'TSNE2'])
    tsne_df[cluster_col] = df[cluster_col].values

    # --- UMAP ---
    reducer = umap.UMAP(n_components=2, random_state=42, n_neighbors=15, min_dist=0.1)
    X_umap = reducer.fit_transform(X_scaled)
    umap_df = pd.DataFrame(X_umap, columns=['UMAP1', 'UMAP2'])
    umap_df[cluster_col] = df[cluster_col].values

    # --- Plotting ---
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    for ax, data, x, y, title in zip(
        axes,
        [pca_df, tsne_df, umap_df],
        ['PCA1', 'TSNE1', 'UMAP1'],
        ['PCA2', 'TSNE2', 'UMAP2'],
        ['Clusters (PCA)', 'Clusters (t-SNE)', 'Clusters (UMAP)']
    ):
        if palette == 'viridis':
            sc = ax.scatter(data[x], data[y], c=data[cluster_col], cmap=palette, s=40, alpha=0.8)
            plt.colorbar(sc, ax=ax, label=cluster_col)
        else:
            sns.scatterplot(
                x=x, y=y, hue=cluster_col, data=data,
                palette=palette, s=40, alpha=0.8, ax=ax, legend=legend_type
            )
        ax.set_title(title, fontsize=12, fontweight='bold')
        ax.grid(True)

    plt.tight_layout()
    plt.show()


In [ ]:
visualize_curr_cluster(full_scaled, clustering_df, cluster_col='KMeans_Cluster')

In [ ]:
visualize_curr_cluster(rfm_scaled, g_rfm, cluster_col='KMeans_Cluster')

In [ ]:
visualize_curr_cluster(pvv_scaled, g_pvv, cluster_col='KMeans_Cluster')

In [ ]:
visualize_curr_cluster(pth_scaled, g_pth, cluster_col='KMeans_Cluster')

## Hierarchical Clustering

`perform_hierarchical_clustering` performs **Agglomerative Hierarchical Clustering** on a scaled feature set. It estimates the **optimal number of clusters** automatically by analyzing the **largest jump in successive merge distances** in the dendrogram. The function then fits a hierarchical clustering model, assigns cluster labels to the dataset, and generates visual summaries including:

* **Dendrogram** with a red dashed line indicating the optimal cluster cut height.
* **Heatmap of cluster profiles** showing the mean values of scaled features for each cluster.

Cluster profiles can also be **inverse-transformed to original scale** for interpretability. The function returns the updated DataFrame with cluster labels, the fitted hierarchical model, and the number of clusters used.


In [ ]:
def perform_hierarchical_clustering(X_scaled, df, features, scaler):
    # Perform hierarchical clustering
    Z = linkage(X_scaled, method='ward')

    # Find optimal number of clusters automatically
    # Distances at each merge
    last = Z[-10:, 2]  # last 10 merge distances
    last_rev = last[::-1]
    idxs = np.arange(1, len(last_rev) + 1)

    # Compute differences between successive merge distances
    diffs = np.diff(last_rev)

    # The largest jump in merge distance
    max_gap_index = np.argmax(diffs) + 1
    n_clusters_hierarchical = max_gap_index + 1  # +1 because clusters = merges+1

    print(f"Optimal number of clusters (hierarchical) based on max distance jump: {n_clusters_hierarchical}")

    # Choose a number of clusters (for comparison, using optimal_k from KMeans if defined)
    n_clusters_to_use = optimal_k  # <-- assuming this is defined globally

    hierarchical_model = AgglomerativeClustering(
        n_clusters=n_clusters_to_use,
        metric='euclidean',
        linkage='ward'
    )

    df['Hierarchical_Cluster'] = hierarchical_model.fit_predict(X_scaled)

    print(f"Hierarchical Clustering with {n_clusters_to_use} clusters complete.")

    # Hierarchical Cluster Profiles (Mean of original scaled features)
    hierarchical_profile_scaled = df.groupby('Hierarchical_Cluster')[features].mean()
    hierarchical_profile_original = pd.DataFrame(
        scaler.inverse_transform(hierarchical_profile_scaled),
        columns=features
    )

    # Dendrogram + Heatmap
    fig, axes = plt.subplots(1, 2, figsize=(18, 6))

    # Dendrogram
    dendrogram(Z, ax=axes[0])
    cut_height = last_rev[max_gap_index]  # height where we cut
    axes[0].axhline(y=cut_height, color='red', linestyle='--')
    axes[0].set_title('Hierarchical Clustering Dendrogram with Optimal Cut')
    axes[0].set_xlabel('Sample Index')
    axes[0].set_ylabel('Distance')

    # Heatmap
    sns.heatmap(hierarchical_profile_scaled.T, annot=True, cmap='viridis', fmt=".2f", ax=axes[1])
    axes[1].set_title('Hierarchical Cluster Profiles (Scaled Feature Means)')
    axes[1].set_xlabel('Cluster')
    axes[1].set_ylabel('Feature')

    plt.tight_layout()
    plt.show()

    return df, hierarchical_model, n_clusters_to_use

In [ ]:
# Full feature set clustering (Hierarchical)
clustering_df, hierarchical_model_full, n_clusters_hier_full = perform_hierarchical_clustering(full_scaled, clustering_df.copy(), features, full_scaler)

In [ ]:
# RFM clustering (Hierarchical)
g_rfm, hierarchical_model_rfm, n_clusters_hier_rfm = perform_hierarchical_clustering(rfm_scaled, g_rfm.copy(), features_g_rfm, rfm_scaler)

In [ ]:
# Purchase Value & Volume clustering (Hierarchical)
g_pvv, hierarchical_model_pvv, n_clusters_hier_pvv = perform_hierarchical_clustering(pvv_scaled, g_pvv.copy(), features_g_pvv, pvv_scaler)

In [ ]:
# Purchase Timing & Habit clustering (Hierarchical)
g_pth, hierarchical_model_pth, n_clusters_hier_pth = perform_hierarchical_clustering(pth_scaled, g_pth.copy(), features_g_pth, pth_scaler)

## DBSCAN Clustering

`perform_dbscan_clustering` applies **Density-Based Spatial Clustering (DBSCAN)** to the scaled dataset. It automatically estimates the **optimal `eps` parameter** using the **k-distance graph and elbow method**. The function then fits a DBSCAN model, assigns cluster labels (with `-1` representing noise points), and provides cluster summaries including:

* **Number of clusters** detected (excluding noise) and **number of noise points**.
* **Cluster profiles** computed as mean values of features for each non-noise cluster, optionally converted back to the original scale for interpretability.

This approach is particularly useful for detecting **arbitrarily-shaped clusters** and identifying **outliers or noise** in the data.

In [ ]:
def perform_dbscan_clustering(X_scaled, df, features, scaler, min_samples=5):
    # Compute k-distances
    neigh = NearestNeighbors(n_neighbors=min_samples)
    nbrs = neigh.fit(X_scaled)
    distances, indices = nbrs.kneighbors(X_scaled)

    # Take the distance to the k-th nearest neighbor
    k_distances = np.sort(distances[:, min_samples-1])

    # Find elbow (eps)
    knee = KneeLocator(range(len(k_distances)), k_distances, curve='convex', direction='increasing')
    eps_optimal = k_distances[knee.knee] if knee.knee is not None else np.percentile(k_distances, 90)

    print(f"Optimal eps (using elbow method): {eps_optimal:.4f}, min_samples={min_samples}")

    # Run DBSCAN with optimal eps
    dbscan_model = DBSCAN(eps=eps_optimal, min_samples=min_samples)
    df['DBSCAN_Cluster'] = dbscan_model.fit_predict(X_scaled)

    # Count clusters and noise
    n_clusters_dbscan = len(set(df['DBSCAN_Cluster'])) - (1 if -1 in df['DBSCAN_Cluster'] else 0)
    n_noise = np.sum(df['DBSCAN_Cluster'] == -1)

    print(f"Number of clusters found: {n_clusters_dbscan}")
    print(f"Number of noise points: {n_noise}")

    # Cluster profiles (exclude noise)
    dbscan_profile = df[df['DBSCAN_Cluster'] != -1].groupby('DBSCAN_Cluster')[features].mean()
    if not dbscan_profile.empty:
        print("\nCluster Profiles (original scale):")
        dbscan_profile_original = pd.DataFrame(
            scaler.inverse_transform(dbscan_profile),
            columns=features,
            index=dbscan_profile.index
        )
        display(dbscan_profile_original.head())
    else:
        print("\nNo meaningful clusters found with current parameters.")

    return df, dbscan_model, n_clusters_dbscan

In [ ]:
clustering_df, dbscan_model, dbscan_clusters = perform_dbscan_clustering(full_scaled, clustering_df.copy(), features, full_scaler)

In [ ]:
# DBSCAN for RFM
g_rfm, dbscan_model_rfm, dbscan_clusters_rfm = perform_dbscan_clustering(rfm_scaled, g_rfm.copy(), features_g_rfm, rfm_scaler)

In [ ]:
# DBSCAN for PVV
g_pvv, dbscan_model_pvv, dbscan_clusters_pvv = perform_dbscan_clustering(pvv_scaled, g_pvv.copy(), features_g_pvv, pvv_scaler)

In [ ]:
# DBSCAN for PTH
g_pth, dbscan_model_pth, dbscan_clusters_pth = perform_dbscan_clustering(pth_scaled, g_pth.copy(), features_g_pth, pth_scaler)

## Gaussian Mixture Model (GMM) Clustering

`perform_gmm_clustering` applies **Gaussian Mixture Model (GMM) clustering** to a scaled dataset. It evaluates multiple numbers of components (clusters) using **BIC (Bayesian Information Criterion)** and **AIC (Akaike Information Criterion)** to determine the optimal number of components. The function then fits a final GMM, assigns cluster labels, and computes cluster profiles including:

* **BIC and AIC plots** for each number of components to visualize model selection.
* **Optimal number of components** based on minimum BIC score.
* **Cluster profiles**, representing the mean values of features for each cluster, optionally inverse-transformed to the original scale for interpretability.

GMM clustering is particularly useful for **modeling clusters with elliptical shapes** and **probabilistic cluster assignments**, allowing soft membership of points to clusters.

In [ ]:
def perform_gmm_clustering(X_scaled, df, features, scaler):
    # Lists to store evaluation metrics
    bic_scores = []
    aic_scores = []
    n_components_range = range(2, 21)

    # Fit GMM for different component numbers
    for n in n_components_range:
        gmm = GaussianMixture(n_components=n, random_state=42, n_init=10)
        gmm.fit(X_scaled)
        bic_scores.append(gmm.bic(X_scaled))
        aic_scores.append(gmm.aic(X_scaled))

    # Plot BIC & AIC scores
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(n_components_range, bic_scores, marker='o', color='blue')
    plt.title('BIC for GMM')
    plt.xlabel('Number of Components')
    plt.ylabel('BIC Score')

    plt.subplot(1, 2, 2)
    plt.plot(n_components_range, aic_scores, marker='o', color='green')
    plt.title('AIC for GMM')
    plt.xlabel('Number of Components')
    plt.ylabel('AIC Score')
    plt.tight_layout()
    plt.show()

    # Choose optimal number of components (where BIC or AIC is lowest)
    optimal_gmm_components = n_components_range[bic_scores.index(min(bic_scores))]
    print(f"Optimal GMM components (based on BIC): {optimal_gmm_components}")

    # Fit final GMM model
    gmm_model = GaussianMixture(n_components=optimal_gmm_components, random_state=42, n_init=10)
    # Use the original df to add the cluster labels
    df['GMM_Cluster'] = gmm_model.fit_predict(X_scaled)


    print(f"\nGMM Clustering with {optimal_gmm_components} components complete.")

    # Cluster profiles in original scale
    # Need to use the features list and scaler here for meaningful output
    gmm_profile_scaled = df.groupby('GMM_Cluster')[features].mean()
    gmm_profile_original = pd.DataFrame(
        scaler.inverse_transform(gmm_profile_scaled),
        columns=features,
        index=gmm_profile_scaled.index
    )

    return df, gmm_model, optimal_gmm_components

In [ ]:
clustering_df, gmm_model, optimal_gmm_components = perform_gmm_clustering(full_scaled, clustering_df.copy(), features, full_scaler)

In [ ]:
rfm_df, gmm_model_rfm, optimal_gmm_components_rfm = perform_gmm_clustering(rfm_scaled, g_rfm.copy(), features_g_rfm, rfm_scaler)

In [ ]:
pvv_df, gmm_model_pvv, optimal_gmm_components_pvv = perform_gmm_clustering(pvv_scaled, g_pvv.copy(), features_g_pvv, pvv_scaler)

In [ ]:
pth_df, gmm_model_pth, optimal_gmm_components_pth = perform_gmm_clustering(pth_scaled, g_pth.copy(), features_g_pth, pth_scaler)

## Affinity Propagation Clustering

`perform_affinity_propagation` applies **Affinity Propagation (AP)** clustering on a scaled dataset. AP identifies **exemplar points** and forms clusters based on **message passing between points**, without requiring a pre-specified number of clusters. The function:

* Fits an **Affinity Propagation model** with user-defined `damping`, `preference`, and `max_iter` parameters.
* Assigns **cluster labels** to the dataset in a new column `AP_Cluster`.
* Computes **cluster profiles** by averaging feature values for each cluster, both in **scaled and original feature space** for interpretability.
* Returns the **number of clusters discovered** automatically by the algorithm.

Affinity Propagation is particularly useful when the **number of clusters is unknown** and clusters may vary in size or density.


In [ ]:
def perform_affinity_propagation(X_scaled, df, features, scaler, damping=0.9, preference=None, max_iter=200):
    # Fit Affinity Propagation
    ap_model = AffinityPropagation(damping=damping, preference=preference, max_iter=max_iter, random_state=42)
    labels = ap_model.fit_predict(X_scaled)

    # Add labels to df
    df['AP_Cluster'] = labels

    n_clusters = len(np.unique(labels))
    print(f"Affinity Propagation found {n_clusters} clusters")

    # Cluster profiles (scaled and original)
    ap_profile_scaled = df.groupby('AP_Cluster')[features].mean()
    ap_profile_original = pd.DataFrame(
        scaler.inverse_transform(ap_profile_scaled),
        columns=features,
        index=ap_profile_scaled.index
    )

    return df, ap_model, n_clusters

In [ ]:
clustering_df, ap_model, n_clusters = perform_affinity_propagation(full_scaled, clustering_df.copy(), features, full_scaler)

In [ ]:
g_rfm, ap_model_rfm, n_clusters_rfm = perform_affinity_propagation(rfm_scaled, g_rfm.copy(), features_g_rfm, rfm_scaler)

In [ ]:
g_pvv, ap_model_pvv, n_clusters_pvv = perform_affinity_propagation(pvv_scaled, g_pvv.copy(), features_g_pvv, pvv_scaler)

In [ ]:
g_pth, ap_model_pth, n_clusters_pth = perform_affinity_propagation(pth_scaled, g_pth.copy(), features_g_pth, pth_scaler)

## MeanShift Clustering

`perform_meanshift` applies **MeanShift clustering** on a scaled dataset. MeanShift identifies clusters by **iteratively shifting points towards regions of high density**, without requiring a predefined number of clusters. The function:

* Estimates an appropriate **bandwidth** automatically using `estimate_bandwidth`.
* Fits a **MeanShift model** and assigns cluster labels to the dataset in a new column `MS_Cluster`.
* Determines the **number of clusters found** and prints it.
* Computes **cluster profiles** as mean values of features for each cluster, transformed back to the **original feature scale** for interpretability.

MeanShift is particularly useful for **discovering clusters of arbitrary shape** and automatically detecting the number of clusters based on **data density**.


In [ ]:
def perform_meanshift(X_scaled, df, features, scaler):
    bandwidth = estimate_bandwidth(X_scaled, quantile=0.2, n_samples=500)
    ms_model = MeanShift(bandwidth=bandwidth)
    clusters = ms_model.fit_predict(X_scaled)
    df['MS_Cluster'] = clusters

    n_clusters = len(set(clusters))
    print(f"Number of clusters found by MeanShift: {n_clusters}")

    profiles = df.groupby('MS_Cluster')[features].mean()
    profiles = pd.DataFrame(scaler.inverse_transform(profiles), index=profiles.index, columns=features)

    return df, ms_model, n_clusters, profiles

In [ ]:
clustering_df, ms_model, n_clusters_ms, profiles_ms = perform_meanshift(full_scaled, clustering_df.copy(), features, full_scaler)

In [ ]:
g_rfm, ms_model_rfm, n_clusters_ms_rfm, profiles_ms_rfm = perform_meanshift(rfm_scaled, g_rfm.copy(), features_g_rfm, rfm_scaler)

In [ ]:
g_pvv, ms_model_pvv, n_clusters_ms_pvv, profiles_ms_pvv = perform_meanshift(pvv_scaled, g_pvv.copy(), features_g_pvv, pvv_scaler)

In [ ]:
g_pth, ms_model_pth, n_clusters_ms_pth, profiles_ms_pth = perform_meanshift(pth_scaled, g_pth.copy(), features_g_pth, pth_scaler)

## Spectral Clustering

`perform_spectral` applies **Spectral Clustering** on a scaled dataset. This method uses the **eigenvalues of a similarity matrix** to reduce the data to a lower-dimensional space before applying a clustering algorithm, typically K-Means. The function:

* Fits a **Spectral Clustering model** with a specified number of clusters (`n_clusters`) and assigns cluster labels in a new column `SP_Cluster`.
* Prints the **number of clusters found**, which is predefined by the user.
* Computes **cluster profiles** by averaging feature values for each cluster, and transforms them back to the **original feature scale** for interpretability.

Spectral Clustering is particularly effective for **detecting clusters with complex, non-convex shapes** and when the data is **not well separated in the original feature space**.


In [ ]:
def perform_spectral(X_scaled, df, features, scaler, n_clusters=3):
    sp_model = SpectralClustering(n_clusters=n_clusters, assign_labels="kmeans", random_state=42)
    clusters = sp_model.fit_predict(X_scaled)
    df['SP_Cluster'] = clusters

    print(f"Number of clusters found by Spectral Clustering: {n_clusters}")

    profiles = df.groupby('SP_Cluster')[features].mean()
    profiles = pd.DataFrame(scaler.inverse_transform(profiles), index=profiles.index, columns=features)

    return df, sp_model, n_clusters, profiles

In [ ]:
clustering_df, sp_model, n_clusters_sp, profiles_sp = perform_spectral(full_scaled, clustering_df.copy(), features, full_scaler)

In [ ]:
# For RFM dataset
g_rfm, sp_model_rfm, n_clusters_sp_rfm, profiles_sp_rfm = perform_spectral(rfm_scaled, g_rfm.copy(), features_g_rfm, rfm_scaler)

In [ ]:
# For PVV dataset
g_pvv, sp_model_pvv, n_clusters_sp_pvv, profiles_sp_pvv = perform_spectral(pvv_scaled, g_pvv.copy(), features_g_pvv, pvv_scaler)

In [ ]:
# For PTH dataset
g_pth, sp_model_pth, n_clusters_sp_pth, profiles_sp_pth = perform_spectral(pth_scaled, g_pth.copy(), features_g_pth, pth_scaler)

## HDBSCAN Clustering

`perform_hdbscan` applies **HDBSCAN (Hierarchical Density-Based Spatial Clustering of Applications with Noise)** on a scaled dataset. HDBSCAN extends DBSCAN by **handling clusters of varying densities** and **identifying noise points** automatically. The function:

* Fits an **HDBSCAN model** with a specified `min_cluster_size` and assigns cluster labels to the dataset in a new column `HDB_Cluster`.
* Computes the **number of clusters found** (excluding noise) and the **number of noise points**.
* Calculates **cluster profiles** as mean feature values for each cluster, ignoring noise points, and transforms them back to the **original feature scale** for interpretability.
* If no meaningful clusters are found, it reports this and returns an empty profile DataFrame.

HDBSCAN is particularly useful for **discovering clusters of varying shapes and densities** and robustly identifying **outliers** in complex datasets.

In [ ]:
def perform_hdbscan(X_scaled, df, features, scaler, min_cluster_size=5):
    hdb_model = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size)
    clusters = hdb_model.fit_predict(X_scaled)
    df['HDB_Cluster'] = clusters

    n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
    n_noise = np.sum(clusters == -1)
    print(f"Number of clusters found by HDBSCAN: {n_clusters}")
    print(f"Number of noise points: {n_noise}")

    profiles = df[df['HDB_Cluster'] != -1].groupby('HDB_Cluster')[features].mean()
    if not profiles.empty:
        profiles = pd.DataFrame(scaler.inverse_transform(profiles), index=profiles.index, columns=features)
    else:
        profiles = pd.DataFrame()
        print("\nNo meaningful clusters found with current parameters.")

    return df, hdb_model, n_clusters, profiles

In [ ]:
clustering_df, hdb_model, n_clusters_hdb, profiles_hdb = perform_hdbscan(full_scaled, clustering_df.copy(), features, full_scaler)

In [ ]:
# For RFM dataset
g_rfm, hdb_model_rfm, n_clusters_hdb_rfm, profiles_hdb_rfm = perform_hdbscan(rfm_scaled, g_rfm.copy(), features_g_rfm, rfm_scaler)

In [ ]:
# For PVV dataset
g_pvv, hdb_model_pvv, n_clusters_hdb_pvv, profiles_hdb_pvv = perform_hdbscan(pvv_scaled, g_pvv.copy(), features_g_pvv, pvv_scaler)

In [ ]:
# For PTH dataset
g_pth, hdb_model_pth, n_clusters_hdb_pth, profiles_hdb_pth = perform_hdbscan(pth_scaled, g_pth.copy(), features_g_pth, pth_scaler)

## OPTICS Clustering

`perform_optics` applies **OPTICS (Ordering Points To Identify the Clustering Structure)** on a scaled dataset. OPTICS is a **density-based clustering algorithm** similar to DBSCAN, but it can **handle varying cluster densities** without requiring a global `eps` parameter. The function:

* Fits an **OPTICS model** with user-defined `min_samples`, `xi`, and `min_cluster_size` parameters.
* Assigns cluster labels to the dataset in a new column `OPTICS_Cluster`, with `-1` indicating noise points.
* Computes the **number of clusters found** (excluding noise) and the **number of noise points**.
* Generates **cluster profiles** by averaging feature values for each non-noise cluster, and transforms them back to the **original feature scale** for interpretability.
* If no meaningful clusters are detected, it reports this and returns an empty profile DataFrame.

OPTICS is particularly useful for **discovering clusters of varying shapes and densities**, while also **robustly detecting outliers** in complex datasets.


In [ ]:
def perform_optics(X_scaled, df, features, scaler, min_samples=5, xi=0.05, min_cluster_size=0.1):
    optics_model = OPTICS(min_samples=min_samples, xi=xi, min_cluster_size=min_cluster_size)
    clusters = optics_model.fit_predict(X_scaled)
    df['OPTICS_Cluster'] = clusters

    n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
    n_noise = np.sum(clusters == -1)
    print(f"Number of clusters found by OPTICS: {n_clusters}")
    print(f"Number of noise points: {n_noise}")

    profiles = df[df['OPTICS_Cluster'] != -1].groupby('OPTICS_Cluster')[features].mean()
    if not profiles.empty:
        profiles = pd.DataFrame(scaler.inverse_transform(profiles), index=profiles.index, columns=features)
    else:
        profiles = pd.DataFrame()
        print("\nNo meaningful clusters found with current parameters.")

    return df, optics_model, n_clusters, profiles

In [ ]:
clustering_df, optics_model, n_clusters_opt, profiles_opt = perform_optics(full_scaled, clustering_df.copy(), features, full_scaler)

In [ ]:
# For RFM dataset
g_rfm, optics_model_rfm, n_clusters_opt_rfm, profiles_opt_rfm = perform_optics(rfm_scaled, g_rfm.copy(), features_g_rfm, rfm_scaler)

In [ ]:
# For PVV dataset
g_pvv, optics_model_pvv, n_clusters_opt_pvv, profiles_opt_pvv = perform_optics(pvv_scaled, g_pvv.copy(), features_g_pvv, pvv_scaler)

In [ ]:
# For PTH dataset
g_pth, optics_model_pth, n_clusters_opt_pth, profiles_opt_pth = perform_optics(pth_scaled, g_pth.copy(), features_g_pth, pth_scaler)

## Birch Clustering

`perform_birch` applies **Balanced Iterative Reducing and Clustering using Hierarchies (BIRCH)** on a scaled dataset. BIRCH is designed for **large datasets** and incrementally builds a **clustering feature tree** to summarize the data before applying global clustering. The function:

* Fits a **BIRCH model** with specified `n_clusters`, `threshold`, and `branching_factor`.
* Assigns cluster labels to the dataset in a new column `Birch_Cluster`.
* Computes the **number of clusters found**.
* Generates **cluster profiles** by averaging feature values for each cluster and transforms them back to the **original feature scale** for interpretability.

BIRCH is particularly useful for **large-scale datasets**, allowing **efficient and incremental clustering** with good scalability and memory usage.


In [ ]:
def perform_birch(X_scaled, df, features, scaler, n_clusters=3, threshold=0.5, branching_factor=50):
    birch_model = Birch(n_clusters=n_clusters, threshold=threshold, branching_factor=branching_factor)
    clusters = birch_model.fit_predict(X_scaled)
    df['Birch_Cluster'] = clusters

    n_clusters = len(set(clusters))
    print(f"Number of clusters found by Birch: {n_clusters}")

    profiles = df.groupby('Birch_Cluster')[features].mean()
    profiles = pd.DataFrame(scaler.inverse_transform(profiles), index=profiles.index, columns=features)

    return df, birch_model, n_clusters, profiles

In [ ]:
clustering_df, birch_model, n_clusters_birch, profiles_birch = perform_birch(full_scaled, clustering_df.copy(), features, full_scaler)

In [ ]:
# For RFM dataset
g_rfm, birch_model_rfm, n_clusters_birch_rfm, profiles_birch_rfm = perform_birch(rfm_scaled, g_rfm.copy(), features_g_rfm, rfm_scaler)

In [ ]:
# For PVV dataset
g_pvv, birch_model_pvv, n_clusters_birch_pvv, profiles_birch_pvv = perform_birch(pvv_scaled, g_pvv.copy(), features_g_pvv, pvv_scaler)

In [ ]:
# For PTH dataset
g_pth, birch_model_pth, n_clusters_birch_pth, profiles_birch_pth = perform_birch(pth_scaled, g_pth.copy(), features_g_pth, pth_scaler)

# Task 7

Visualize clustering results using different dimensionality reduction and visualization techniques. Apply PCA for cluster visualization in reduced dimensions. Create feature-based visualizations without PCA. Use t-SNE and UMAP for advanced non-linear visualizations to better interpret cluster structures.


## PCA-Based Cluster Visualization

`visualize_clusters_pca` provides a **2D visualization of clustering results** by projecting the scaled dataset onto the first two **principal components (PCA)**. The function:

* Automatically detects all **cluster columns** in the DataFrame ending with `_Cluster`.
* Handles **noise-based clustering methods** such as DBSCAN, HDBSCAN, and OPTICS, displaying noise points (`-1`) in gray.
* Plots **scatterplots of PC1 vs PC2** for each cluster column, using a **distinct color palette** for each cluster.
* Supports **multiple clustering results simultaneously**, arranging up to three plots per row and removing unused subplot axes.
* Enhances interpretability by showing **cluster separations** in reduced 2D space, making it easy to compare different clustering algorithms visually.

This function is particularly useful for **quickly inspecting cluster structure**, identifying overlapping clusters, and evaluating **how well the clustering separates the data** in a low-dimensional representation.


In [ ]:
def visualize_clusters_pca(X_scaled, df):
    # PCA transformation
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    X_pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2'], index=df.index)

    # Automatically extract cluster columns (ending with "_Cluster")
    cluster_columns = [col for col in df.columns if col.endswith("_Cluster")]

    if not cluster_columns:
        raise ValueError("No cluster columns found in the dataframe ending with '_Cluster'")
    available_cols = [col for col in cluster_columns if col in df.columns]
    for col in available_cols:
        X_pca_df[col] = df[col]

    # Plot settings
    n_plots = len(available_cols)
    n_rows = (n_plots + 2) // 3  # up to 3 per row
    fig, axes = plt.subplots(n_rows, 3, figsize=(18, 5 * n_rows))
    axes = axes.flatten()

    for ax, col in zip(axes, available_cols):
        unique_clusters = sorted(X_pca_df[col].unique())
        n_clusters = len(unique_clusters)

        # Handle noise (cluster = -1)
        if -1 in unique_clusters:
            colors = [(0.6, 0.6, 0.6)] + sns.color_palette("viridis", n_clusters - 1)
            palette = dict(zip(unique_clusters, colors))
        else:
            palette = sns.color_palette("viridis", n_clusters)

        sns.scatterplot(
            data=X_pca_df, x='PC1', y='PC2', hue=col,
            palette=palette, s=70, alpha=0.85,
            edgecolor='black', linewidth=0.3, ax=ax
        )
        ax.set_title(f"{col.replace('_Cluster','')} Clusters", fontsize=14, fontweight='bold')
        ax.legend().remove()

    # Remove unused subplots if any
    for i in range(len(available_cols), len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

In [ ]:
visualize_clusters_pca(full_scaled, clustering_df)

In [ ]:
visualize_clusters_pca(rfm_scaled, g_rfm)

In [ ]:
visualize_clusters_pca(pvv_scaled, g_pvv)

In [ ]:
visualize_clusters_pca(pth_scaled, g_pth)

## Feature-Based Cluster Visualization (No PCA)

`visualize_clusters_feature_based` provides a **2D visualization of clustering results using two actual features** from the dataset, without applying PCA. The function:

* Automatically detects all **cluster columns** in the DataFrame ending with `_Cluster`.
* Handles **noise-based clustering methods** such as DBSCAN, HDBSCAN, and OPTICS, showing noise points (`-1`) in gray.
* Plots **scatterplots of the two selected features** for each cluster column, using distinct colors for each cluster.
* Supports **multiple clustering results simultaneously**, arranging up to three plots per row and removing unused subplot axes.
* Enhances interpretability by allowing **direct inspection of cluster structure in the original feature space**, helping to understand how clusters relate to actual feature values.

This function is particularly useful for **comparing clustering results on real features** and evaluating cluster separation without dimensionality reduction.


In [ ]:
def visualize_clusters_feature_based(df, features):
    # Choose two features for visualization
    f1, f2 = features[0], features[1]

    # Automatically extract cluster columns (ending with "_Cluster")
    cluster_columns = [col for col in df.columns if col.endswith("_Cluster")]

    if not cluster_columns:
        raise ValueError("No cluster columns found in the dataframe ending with '_Cluster'")

    available_cols = [col for col in cluster_columns if col in df.columns]
    n_plots = len(available_cols)
    n_rows = (n_plots + 2) // 3  # up to 3 per row

    fig, axes = plt.subplots(n_rows, 3, figsize=(18, 5 * n_rows))
    axes = axes.flatten()

    for ax, col in zip(axes, available_cols):
        unique_clusters = sorted(df[col].unique())
        n_clusters = len(unique_clusters)

        # Handle noise (-1)
        if -1 in unique_clusters:
            colors = [(0.6, 0.6, 0.6)] + sns.color_palette("viridis", n_clusters - 1)
            palette = dict(zip(unique_clusters, colors))
        else:
            palette = sns.color_palette("viridis", n_clusters)

        sns.scatterplot(
            data=df, x=f1, y=f2, hue=col,
            palette=palette, s=70, alpha=0.85,
            edgecolor='black', linewidth=0.3, ax=ax
        )
        ax.set_title(f"{col.replace('_Cluster','')} Clusters", fontsize=14, fontweight='bold')
        ax.legend().remove()

    # Remove unused subplots if any
    for i in range(len(available_cols), len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()


In [ ]:
visualize_clusters_feature_based(clustering_df, features=["Recency", "Frequency"])

In [ ]:
visualize_clusters_feature_based(g_rfm, ["Recency", "Frequency"])

In [ ]:
visualize_clusters_feature_based(g_pvv, ["Monetary", "Total_Items_Purchased"])

In [ ]:
visualize_clusters_feature_based(g_pth, ["Recency", "Avg_Days_Between_Purchases"])

## t-SNE-Based Cluster Visualization

`visualize_clusters_tsne` provides a **2D visualization of clustering results using t-SNE** on the scaled feature set. The function:

* Automatically detects all **cluster columns** in the DataFrame ending with `_Cluster`.
* Applies **t-SNE** to reduce the high-dimensional scaled features to two dimensions (`TSNE1` and `TSNE2`) for visualization.
* Plots **scatterplots for each cluster column** in a grid layout, using a distinct color palette (`tab10`) for clusters.
* Supports **multiple clustering results simultaneously**, arranging subplots in rows and columns for better readability.
* Enhances interpretability by showing **non-linear relationships** and **cluster separations** in a low-dimensional t-SNE projection.

This function is particularly useful for **visualizing complex cluster structures**, especially when clusters are not linearly separable in the original feature space.


In [ ]:
def visualize_clusters_tsne(df, X_scaled):
    # Automatically extract cluster columns (ending with "_Cluster")
    cluster_columns = [col for col in df.columns if col.endswith("_Cluster")]

    if not cluster_columns:
        raise ValueError("No cluster columns found in the dataframe ending with '_Cluster'")

    tsne = TSNE(n_components=2, random_state=42, perplexity=30, learning_rate=200)
    X_tsne = tsne.fit_transform(X_scaled)

    tsne_df = pd.DataFrame(X_tsne, columns=['TSNE1', 'TSNE2'], index=df.index)

    # Merge with clustering results
    tsne_df = pd.concat([tsne_df, df[cluster_columns]], axis=1)

    # Plot all clusters in a grid
    n_clusters = len(cluster_columns)
    n_cols = 3
    n_rows = (n_clusters + n_cols - 1) // n_cols

    plt.figure(figsize=(6*n_cols, 5*n_rows))

    for i, col in enumerate(cluster_columns, 1):
        ax = plt.subplot(n_rows, n_cols, i)
        sns.scatterplot(
            data=tsne_df,
            x='TSNE1', y='TSNE2',
            hue=col,
            palette='tab10',
            s=40, alpha=0.8, edgecolor=None,
            legend=False
        )
        ax.set_title(f"t-SNE: {col}")

    # Increase spacing between subplots
    plt.subplots_adjust(wspace=0.25, hspace=0.35)

    plt.show()

In [ ]:
visualize_clusters_tsne(clustering_df, full_scaled)

In [ ]:
visualize_clusters_tsne(g_rfm, rfm_scaled)

In [ ]:
visualize_clusters_tsne(g_pvv, pvv_scaled)

In [ ]:
visualize_clusters_tsne(g_pth, pth_scaled)

## UMAP-Based Cluster Visualization

`visualize_clusters_umap` provides a **2D visualization of clustering results using UMAP** on the scaled feature set. The function:

* Automatically detects all **cluster columns** in the DataFrame ending with `_Cluster`.
* Applies **UMAP** to reduce the high-dimensional scaled features to two dimensions (`UMAP1` and `UMAP2`) while preserving **local and global data structure**.
* Plots **scatterplots for each cluster column** in a grid layout, using a distinct color palette (`tab10`) for clusters.
* Supports **multiple clustering results simultaneously**, arranging subplots in rows and columns for readability.
* Enhances interpretability by showing **cluster separations and relationships** in a low-dimensional UMAP projection, which is particularly effective for **non-linear structures** in the data.

This function is especially useful for **visualizing complex, high-dimensional clusters** while maintaining both **local neighborhood relationships** and overall cluster topology.

In [ ]:
def visualize_clusters_umap(df, X_scaled):
    # Automatically extract cluster columns (ending with "_Cluster")
    cluster_columns = [col for col in df.columns if col.endswith("_Cluster")]

    reducer = umap.UMAP(n_components=2, random_state=42, n_neighbors=15, min_dist=0.1)
    X_umap = reducer.fit_transform(X_scaled)

    umap_df = pd.DataFrame(X_umap, columns=['UMAP1', 'UMAP2'], index=df.index)

    # Merge with clustering results
    umap_df = pd.concat([umap_df, df[cluster_columns]], axis=1)

    # Plot all clusters in a grid
    n_clusters = len(cluster_columns)
    n_cols = 3
    n_rows = (n_clusters + n_cols - 1) // n_cols

    fig = plt.figure(figsize=(6*n_cols, 5*n_rows))

    for i, col in enumerate(cluster_columns, 1):
        ax = plt.subplot(n_rows, n_cols, i)
        sns.scatterplot(
            data=umap_df,
            x='UMAP1', y='UMAP2',
            hue=col,
            palette='tab10',
            s=40, alpha=0.8, edgecolor=None,
            legend=False
        )
        ax.set_title(f"UMAP: {col}", fontsize=12, fontweight="bold")

    # Increase spacing between subplots
    plt.subplots_adjust(wspace=0.3, hspace=0.4)
    plt.show()

In [ ]:
visualize_clusters_umap(clustering_df, full_scaled)

In [ ]:
visualize_clusters_umap(g_rfm, rfm_scaled)

In [ ]:
visualize_clusters_umap(g_pvv, pvv_scaled)

In [ ]:
visualize_clusters_umap(g_pth, pth_scaled)

# Task 8

Evaluate the performance of clustering algorithms using metrics such as Silhouette Score, Calinski–Harabasz Index, and Davies–Bouldin Index. Summarize the results in a comparison table and interpret which clustering method provides the most meaningful segmentation.


## Clustering Evaluation Metrics Table

`clustering_metrics_table` generates a **comparative table of clustering evaluation metrics** for all cluster assignments in a dataset. The function:

* Automatically detects all **cluster columns** in the DataFrame ending with `_Cluster`.
* Computes three commonly used clustering metrics for each algorithm:

  * **Silhouette Score** – measures how similar points are within a cluster compared to other clusters (higher is better).
  * **Calinski-Harabasz Index** – evaluates cluster dispersion; higher values indicate well-separated clusters.
  * **Davies-Bouldin Index** – evaluates cluster similarity; lower values indicate better clustering.
* Handles **noise-based clustering algorithms** like DBSCAN, HDBSCAN, and OPTICS, where all points might be noise (`-1`), marking metrics as `"N/A"` if computation is not possible.
* Returns a **DataFrame summarizing metrics for each clustering algorithm**, making it easy to compare cluster quality across different methods.

In [ ]:
def clustering_metrics_table(X_scaled, df):
    # Automatically extract cluster columns (ending with "_Cluster")
    cluster_columns = [col for col in df.columns if col.endswith("_Cluster")]

    metrics = []

    for col in cluster_columns:
        labels = df[col].values

        # Skip if clustering failed (e.g., all -1 in DBSCAN/HDBSCAN)
        if len(set(labels)) <= 1 or (len(set(labels)) == 2 and -1 in labels):
            metrics.append({
                "Algorithm": col,
                "Silhouette": "N/A",
                "Calinski-Harabasz": "N/A",
                "Davies-Bouldin": "N/A"
            })
            continue

        try:
            silhouette = silhouette_score(X_scaled, labels)
            calinski = calinski_harabasz_score(X_scaled, labels)
            davies = davies_bouldin_score(X_scaled, labels)
        except Exception as e:
            silhouette, calinski, davies = "Err", "Err", "Err"

        metrics.append({
            "Algorithm": col,
            "Silhouette": round(silhouette, 4) if isinstance(silhouette, float) else silhouette,
            "Calinski-Harabasz": round(calinski, 2) if isinstance(calinski, float) else calinski,
            "Davies-Bouldin": round(davies, 4) if isinstance(davies, float) else davies
        })

    metrics_df = pd.DataFrame(metrics)
    return metrics_df

In [ ]:
metrics_clustering_df = clustering_metrics_table(full_scaled, clustering_df)
print("\n--- Clustering Evaluation Metrics ---\n")
display(metrics_clustering_df.style.set_table_styles(
    [{'selector': 'th', 'props': [('font-size', '12pt'), ('text-align', 'center')]},
     {'selector': 'td', 'props': [('text-align', 'center')]}]
))

In [ ]:
metrics_g_rfm = clustering_metrics_table(rfm_scaled, g_rfm)
print("\n--- Clustering Evaluation Metrics ---\n")
display(metrics_g_rfm.style.set_table_styles(
    [{'selector': 'th', 'props': [('font-size', '12pt'), ('text-align', 'center')]},
     {'selector': 'td', 'props': [('text-align', 'center')]}]
))

In [ ]:
metrics_g_pvv = clustering_metrics_table(full_scaled, g_pvv)
print("\n--- Clustering Evaluation Metrics ---\n")
display(metrics_g_pvv.style.set_table_styles(
    [{'selector': 'th', 'props': [('font-size', '12pt'), ('text-align', 'center')]},
     {'selector': 'td', 'props': [('text-align', 'center')]}]
))

In [ ]:
metrics_g_pth = clustering_metrics_table(full_scaled, g_pth)
print("\n--- Clustering Evaluation Metrics ---\n")
display(metrics_g_pth.style.set_table_styles(
    [{'selector': 'th', 'props': [('font-size', '12pt'), ('text-align', 'center')]},
     {'selector': 'td', 'props': [('text-align', 'center')]}]
))

In [ ]:
clustering_df.to_csv('clustering_df.csv', index=False)
g_rfm.to_csv('g_rfm.csv', index=False)
g_pvv.to_csv('g_pvv.csv', index=False)
g_pth.to_csv('g_pth.csv', index=False)
